# **Let's start to automate some translation task for Magic: the Gathering translation**

> *A bit of context*: every weekend, a group of passionate and cool French judges work for about 1H30 (for free), on translating the newest article on the (awesome) ✨ [**Cranial Insertion** Magic: the Gathering blog](https://www.cranial-insertion.com/), from 🇺 English to 🇫 French.
> I want to automate this task, as much as possible!
> *Let's work more now, to work less in the future!*

Here is what I would like to obtain: an entirely automated pipeline that takes care of helping us to produce the best translation of these English-written articles covering our favorite game (Magic!), in order to help us thanks to coding and AI, and which helps us be more efficient.

I'm thinking about these steps:
1. when one of the North-American author submits their weekly article on Cranial Insertion, something should automatically trigger a script running "on the cloud" (a free/cheap "Google Lambda function" ?) or on the Cranial Insertion hosting server itself,
2. this script should do [like I did last year](https://github.com/Naereen/Assistant-traduction-CranialInsertion-en-Python/blob/master/import-the-latest-article.ipynb), by connecting to the Cranial Insertion admin webpage and download the raw text of the newest article. Then it could be sent to the "cloud", into a new tab in our shared [Google doc](https://docs.google.com/), named by the date of the day.
3. Ideally, this Google doc should be cut in three equally-spacen parts, each with a different color.
4. Then, I want to also do [like I did last year in June](https://github.com/Naereen/Assistant-traduction-CranialInsertion-en-Python/blob/master/translate-card-names.ipynb), to translate as much as possible the names of the Magic cards when they are in tags like `[c]Card Name[/c]` or [c=Card Name]Card Name[/c]`.
5. If possible, a free and efficient LLM specialized on translation from English to French, like HuggingFace's [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) model, could be used to suggest a first draft of a translation, paragraph by paragraph, to help us translate the article.
6. Then when this is done and our shared Google doc has been updated to contain what should be read and translated, a message could be posted to our Discord server automatically, to notify the team of translator that the newest article is ready for translation! See <https://www.makerhacks.com/python-messages-discord/>, which requires a [Discord webhook](https://support.discord.com/hc/en-us/articles/228383668-Intro-to-Webhooks).
   > If all this works fine, we could improve it by automatically affecting three out of the six translators, to a third of the article's content, and notifying only them in the message sent to Discord.

*Note:*
- If running this task automatically can't be done easily, I can write a notebook that can run on Google colab or locally on my machine, and run it manually each week.
- From Google colab, [it is (of course) possible to save a document to Google Drive](https://stackoverflow.com/questions/64808087/how-do-i-save-files-from-google-colab-to-google-drive#64810963), so it must be possible to edit a Google doc.

> More links?
> - See <https://huggingface.co/docs/transformers/tasks/translation> for the documentation I followed initially.
> - See <https://github.com/Naereen/Assistant-traduction-CranialInsertion-en-Python> for the initial project I wrote (documentation in French) in June 2024.

## Small signature and versions of some modules

### `requirements.txt`

```
watermark
sacremoses
tensorflow
transformers
feedparser
chardet
requests
beautifulsoup4
tqdm
nest_asyncio
scrython
mtgsdk
googleapiclient
google
```

In [3]:
!pip install watermark sacremoses tensorflow transformers feedparser requests beautifulsoup4 tqdm scrython nest_asyncio mtgsdk chardet

In [4]:
%load_ext watermark
%watermark -v -m -a "Lilian Besson (Naereen)" -gu "Naereen" -g -p tensorflow,transformers,feedparser,requests,bs4,tqdm,scrython,nest_asyncio,mtgsdk,chardet

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Lilian Besson (Naereen)

Github username: Naereen

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

tensorflow  : 2.18.0
transformers: 4.48.3
feedparser  : 6.0.11
requests    : 2.32.3
bs4         : 4.13.3
tqdm        : 4.67.1
scrython    : 1.11.0
nest_asyncio: 1.6.0
mtgsdk      : 1.3.1
chardet     : 5.2.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

Git hash: 



----

## Using Google's colab GPU and setting up the requirements to translate English-to-French
Let's check that the Google Colab notebook runs on a GPU and not a CPU, if possible. Otherwise, inference (translation English to French) will be slower.

In [6]:
running_on_GPU = False

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print("GPU device not found, running on the CPU (20/30x slower for LLM/NLP/ML)")
else:
    print('Found GPU at: {}'.format(device_name))
    running_on_GPU = True

GPU device not found, running on the CPU (20/30x slower for LLM/NLP/ML)


Let's use this model <https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr> which is specialized in the translation of natural text written in English, into French.
I'll study later on how we could finetune this model to improve its performance on documents related to Magic: the Gathering.

In [7]:
%%time
from transformers import pipeline
pipe_en2fr = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-fr")

Device set to use cpu


CPU times: user 9.2 s, sys: 2.99 s, total: 12.2 s
Wall time: 16.1 s


We can then very quickly write a function to translate text, and use it on small sentences:

In [8]:
def en2fr_small(text):
    return pipe_en2fr(text)[0]['translation_text']

In [9]:
%%time
print(en2fr_small("The Portuguese teacher is very demanding."))
# expected output: Le professeur de portugais est très exigeant.

Le professeur de portugais est très exigeant.
CPU times: user 1.33 s, sys: 6.26 ms, total: 1.34 s
Wall time: 1.33 s


In [10]:
%%time
print(en2fr_small("Who are you? I am a LLM running on the Google cloud (Google colab)."))
# expected output: Qui êtes-vous? Je suis un LLM fonctionnant sur le cloud Google (Google colab).

Qui êtes-vous? Je suis un LLM fonctionnant sur le cloud Google (Google colab).
CPU times: user 4.19 s, sys: 13.4 ms, total: 4.2 s
Wall time: 4.29 s


I'll print paragraphs using this code:

In [11]:
from IPython.display import display, Markdown
def print_paragraph(paragraph):
    return display(Markdown(f"> {paragraph}"))

-----

## Splitting a paragraph on sentences, translating each of them, and joining the result

Let's improve the `en2fr` translation function:

In [13]:
def en2fr(text, split_length=351):
    """ Translate <text> from English to French.

    EXPERIMENTAL: If its size is larger than <split_length>, split it by sentences, translate each separately, then join the result.
    """
    if len(text) < split_length:
        return pipe_en2fr(text)[0]['translation_text']
    else:
        sentences = text.split(".")
        translated_sentences = []
        for sentence in sentences:
            if len(sentence) > 0:
                text = sentence
                translated_text = pipe_en2fr(text)[0]['translation_text']
                translated_sentences.append(translated_text)
        return ".".join(translated_sentences)

This small hack I wrote seems to work efficiently, to translate longer texts!

--------

## Trying to translate automatically a first paragraph from a recent Cranial Insertion article

In [13]:
%%time
first_paragraph = "[a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]"
print("The first paragraph in English is:")
print_paragraph(first_paragraph)

first_paragraph_fr = en2fr(first_paragraph)
print("The first paragraph in French is:")
print_paragraph(first_paragraph_fr)

The first paragraph in English is:


> [a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]

The first paragraph in French is:


> [a]La chasse au dragon glaciale en tant que sort n'a pas de cible, donc vous n'annoncez pas de cible au moment où vous la lancez. Si vous choisissez de défausser une carte pendant sa résolution et que cette carte s'avère être une carte non terrestre, cela déclenche une capacité déclenchée par réflexe qui a une cible, et vous annoncez sa cible lorsque vous mettez cette capacité sur la pile.

CPU times: user 14.2 s, sys: 111 ms, total: 14.3 s
Wall time: 14.4 s


This first try of translating a paragraph from [a recent Cranial Insertion article](https://www.cranial-insertion.com/article/4380) is quite a success!
**It's very quick: about 1 second to translate a paragraph, when running on a free GPU in Google Colab!**

> CPU times: user 765 ms, sys: 1.34 ms, total: 766 ms
> Wall time: 765 ms

**It is much slower on a CPU (as expected), about 20 times slower:**

> CPU times: user 13.8 s, sys: 161 ms, total: 13.9 s
> Wall time: 16.1 s

In [ ]:
ratio_cpu_to_gpu = round(16.1 / 0.765, 3)
print(f"It is approximately {ratio_cpu_to_gpu} times slower to use the (entirely Free) CPU than to use the (not-free) cheapest GPU.")

It is approximately 21.046 times slower to use the (entirely Free) CPU than to use the (not-free) cheapest GPU.


I'm pretty certain that this model cannot translate paragraph which are longer than about 370 caracters, from what I tried.
See above, I wrote an improved version of `en2fr`, to split paragraphs into sentences, and translate them one-by-one, if the paragraph is longer than the max lengths.

On my first experiment, I was using a Google T4 GPU, and:
It took about 23 seconds to generate a first draft of a translation for a content lengthy like two thirds (2/3) of a usual-size article.
So it should take about 35 seconds to translate a full-size article, that's great!

------

## Failures of the [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) model for translating text about Magic: the Gathering (on Cranial Insertion)

Here are a few points I observed on which this great NLP LLM fails:

- land/nonland => ~~terrestre~~ terrain/non terrain
- you => ~~vous~~ tu (on tutoie le lecteur dans les articles Cranial)
- reflexive trigger => capacité déclenchée ~~par réflexe~~ réflexive
- and most likely MANY MORE!

### TODO: check more examples to list here the failures I observed

### Future goal: fine-tune this model 🪄
One future goal will be to fine-tune the [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) model, using the data of all the previously manually translated Cranial Insertion articles (and maybe other sources), in order to fix these issues and improve the overall quality of the translation.

------

## Getting the text of the latest article

By using the RSS feed of the Cranial Insertion blog, I'll download the raw content of the newest article.

In [14]:
import feedparser

In [15]:
rss_url = "https://www.cranial-insertion.com/rss"
print(f"Reading the feed at {rss_url}...")

blog_feed = feedparser.parse(rss_url)
print(f"Succesfully read the feed at {rss_url}, named « {blog_feed.feed.title} »")

Reading the feed at https://www.cranial-insertion.com/rss...
Succesfully read the feed at https://www.cranial-insertion.com/rss, named « Cranial Insertion »


In [16]:
num_article = 0

latest_article_link = blog_feed.entries[num_article].link
print("- The latest article is here:", latest_article_link)

latest_article_title = blog_feed.entries[num_article].title
print("- Its title is:", latest_article_title)

latest_article_date = blog_feed.entries[num_article].published
print("- It was published on:", latest_article_date)

article_id = latest_article_link.split("/")[-1]
print("- The article ID is:", article_id)

- The latest article is here: http://www.cranial-insertion.com/article/4380
- Its title is: Cranial Insertion: Cloudy With a Chance of Dragonstorms
- It was published on: Mon, 07 Apr 2025 00:00:00 EDT
- The article ID is: 4380


In [17]:
def get_latest_article_id(num_article=0):
    blog_feed = feedparser.parse(rss_url)
    latest_article_link = blog_feed.entries[num_article].link
    article_id = latest_article_link.split("/")[-1]
    return article_id

print(f"The latest article ID is: {get_latest_article_id()}")

The latest article ID is: 4380


In [18]:
try:
    with open("cranial-insertion.com.cookies", 'r') as file:
        cranial_insertion_cookie = file.read()
    print("Cranial Insertion cookie successfully loaded from local file 'cranial-insertion.com.cookies', it can now be used (it's a secret)")
except FileNotFoundError:
    from google.colab import userdata
    cranial_insertion_cookie = userdata.get('CI_COOKIE')
    print("Cranial Insertion cookie successfully loaded from Google Colab secrets, it can now be used (it's a secret)")

Cranial Insertion cookie successfully loaded from Google Colab secrets, it can now be used (it's a secret)


We need two modules, to get the text of Cranial Insertion articles:

In [19]:
import requests
from bs4 import BeautifulSoup

In [83]:
def get_article_text(article_id=article_id, latest_article_edit_url=None):
    if not latest_article_edit_url:
        latest_article_edit_url = f"https://www.cranial-insertion.com/staff/articles/{article_id}/edit"
    print(f"Reading the article at URL {latest_article_edit_url} ...")

    cookies = {
        'loggedin': cranial_insertion_cookie,
        'siteLang': 'fr',
    }
    print(f"Using French language and my editor cookie...")

    response = requests.get(latest_article_edit_url, cookies=cookies)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"The web page has been read and it's title is « {soup.title} » !")

    latest_article_text = soup.find(id="thisArticleText").get_text()
    latest_article_title = soup.find(id="thisArticleTitle").get('value')
    latest_article_date = soup.find(id="thisArticlePubdate").get('value')

    return latest_article_text, latest_article_title, latest_article_date, soup

We can know get the text of any English article, if we know its ID:

> I could improve that but there is no need.

In [84]:
list_of_article_id = [
    4380,
    4376,
    4374,
    4371,
    # etc, there is 20 years of weekly articles!
    # see https://www.cranial-insertion.com/archive for a list
]

article_id = list_of_article_id[-1]
try:
    latest_article_text, latest_article_title, latest_article_date, soup = get_article_text(article_id)
except:
    article_id = get_latest_article_id()
    latest_article_text, latest_article_title, latest_article_date, soup = get_article_text(article_id)

latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"
print("- The latest article is here:", latest_article_link)
print("- Its title is:", latest_article_title)
print("- Its publication date is:", latest_article_date)
print(f"- This article {article_id} raw text is:")
display(Markdown(latest_article_text))

Reading the article at URL https://www.cranial-insertion.com/staff/articles/4371/edit ...
Using French language and my editor cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !
- The latest article is here: https://www.cranial-insertion.com/article/4371
- Its title is: A Very Green Day
- Its publication date is: 03/17/2025
- This article 4371 raw text is:


[cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.

As always, if you have questions for us, you can email them to us at [email]moko@cranialinsertion.com[/email], or tweet short questions to our X account @CranialTweet. One of our authors will reply to you, and your question might appear in a future article.

[hr]
[q]Let's say I control [c]Hardened Scales[/c] and I cast [c]Bounty of the Hunt[/c], assigning one counter to each of three target creatures. Because of Hardened Scales, this will result in two counters being put onto each target. Do I have to remove all six counters or just one counter per creature at the beginning of the cleanup step?[/q]

[a]I'm afraid you'll have to remove all six counters. Bounty of the Hunt tracks how many counters each creature received "this way", which means how many counters it received due to following the instructions of Bounty of the Hunt as modified by any applicable replacement effects. Since each target creature received two counters this way, you'll have to remove two counters from each of them.[/a]

[hr]
[q]If I control [c]Spelunking[/c] and [c]Titania, Gaea Incarnate[/c] enters, do the lands return from the graveyard tapped or untapped?[/q]

[a]The lands will enter untapped unless the land itself has a replacement effect that says it enters tapped, in which case you get a choice between tapped and untapped. Titania's effect instructing you to return the lands to the battlefield tapped is not a replacement effect, it's just the effect of what the ability does. If the land doesn't have a replacement effect itself, then Spelunking's effect is the only replacement effect involved, and it replaces Titania's "tapped" with "untapped". If the land has a replacement effect along the lines of "this land enters tapped", then you choose the order in which to apply the land's replacement effect and Spelunking's effect, which results in a choice between tapped and untapped.[/a]

[hr]
[q]If I control [c]Ashaya, Soul of the Wild[/c], can I put creatures onto the battlefield with [c]Cultivator Colossus[/c]'s ability because they're also lands?[/q]

[a]No, that doesn't work. Ashaya's ability only affects nontoken creatures that are on the battlefield. It does not affect the creature cards that are in your hand.[/a]

[hr]
[q]I control a [c]Plaxcaster Frogling[/c] that still has a +1/+1 counter left on it and my opponent tries to [c]Murder[/c] it. If I activate the Frogling's ability to give it shroud, does that make Murder fizzle?[/q]

[a]Absolutely. Plaxcaster Frogling can target itself with its ability, and the ability resolves first, giving the Frogling shroud. When Murder tries to resolve, it finds that its only target has become an illegal target, so it doesn't resolve and just goes to the graveyard.[/a]

[hr]
[q]If I use [c]Biophagus[/c]'s mana to cast [c]Nylea, God of the Hunt[/c] but I don't have enough devotion to green for Nylea to be a creature, does it still get the +1/+1 counter?[/q]

[a]Yup! Since Nylea is by default a creature and the ability that turns off its creature-ness only functions on the battlefield, you are in fact using Biophagus's mana to cast a creature spell, so Nylea gets the replacement effect "enters with an additional +1/+1 counter" stapled onto it. There is nothing stopping a noncreature permanent from entering with a +1/+1 counter, and there is no reason for a +1/+1 counter to fall off from a noncreature permanent. The counter will be placed on Nylea and stays there, it just won't have any effect until Nylea is a creature.[/a]

[hr]
[q]If [c]Displaced Dinosaurs[/c] makes a planeswalker creature and that planeswalker creature gets attacked, can it block the attacker?[/q]

[a]Sure, that's perfectly legal and probably beneficial even though it does not change the type or amount of harm that's coming to the planeswalker. Whether you block the attacker or not, it will deal its combat damage to the planeswalker creature, and in either case this will cause the planeswalker creature to lose loyalty counters (because it's a planeswalker) and to have damage marked on it (because it's a creature). However, as a blocking creature the planeswalker creature gets to deal combat damage back to the attacker, and as a 7/7 Dinosaur it deals a pretty significant amount of damage.[/a]

[hr]
[cleft=Wild Dogs]Hear the dogs howling out of key
To a hymn called "Faith and Misery"[/cleft][q]What does [c]Wild Dogs[/c]'s triggered ability do in Two-Headed Giant?[/q]

[a]It does very little. One might think that a player on the team with the greater life total would gain control of the Dogs, but that's not how this works. The ability checks before it even goes on the stack whether an individual [i]player[/i] has more life than each other individual player. When an effect in Two-Headed Giant asks for a player's life total, the answer is that player's team's shared life total, so it will always get the same number for the two players on the same team. This means that the condition the ability looks for can never be true in Two-Headed Giant, so the ability never goes on the stack.[/a]

[hr]
[q]What are the card types that [c]Winter, Cynical Opportunist[/c] looks for?[/q]

[a]The term "card type" refers to the "main" card type of a card, which includes things like artifact, creature, land, and so on. On the type line there may be a supertype such as legendary or basic in front of the card type; those are not card types, so Winter isn't interested in those. After the dash you'll find subtypes such as Elf, Forest, or Equipment. Those aren't card types either, so Winter isn't interested in those, either.[/a]

[hr]
[q]Can [c]Winter, Cynical Opportunist[/c]'s ability put a battle's back face onto the battlefield? For example, can I put [c]Invasion of Ikoria[/c] onto the battelfield as [c]Zilortha, Apex of Ikoria[/c]?[/q]

[a]No. When an effect puts a double-faced card onto the battlefield without specifying a face, it is put onto the battlefield with its front face up.[/a]

[hr]
[q]Does [c]Garruk's Packleader[/c] see itself entering?[/q]

[a]A little bit yes, but mostly no. Every creature sees itself entering the battlefield, since enters abilities are checked after the event, when the creature is on the battlefield. However, Garruk's Packleader's ability only triggers whenever [i]another[/i] creature with power 3 or greater enters, so it doesn't trigger its own enters ability.[/a]

[hr]
[q]I'm in a game of Commander, my [c]Ms. Bumbleflower[/c]'s ability has triggered, but all my opponents have hexproof. Can I still give a creature a +1/+1 counter and flying?[/q]

[a]No. In order to put Ms. Bumbleflower's ability on the stack, you have to choose legal targets for each required target. The ability needs a target opponent and a target creature, and since you can't legally target an opponent, the ability is immediately removed from the stack.[/a]

[hr]
[q]I am casting [c]Reborn Hope[/c] off of a [c]Counterpoint[/c] that targeted a spell with mana value 2 or greater. Can Reborn Hope bring back this Counterpoint?[/q]

[a]No. You have to choose the target for Reborn Hope at the moment you cast it, and you're casting Reborn Hope while Counterpoint is resolving. At that moment, Counterpoint is on the stack, not in the graveyard, so you can't target it with Reborn Hope.[/a]

[hr]
[cright=Deathrite Shaman]I pledge allegiance to the underworld[/cright][q]Does [c]Niv-Mizzet, Guildpact[/c] count hybrid one-drops such as [c]Judge's Familiar[/c] or [c]Deathrite Shaman[/c] towards its X?[/q]

[a]Absolutely. The hybrid mana symbol has both of its colors, which means that the permanent is both of those colors. Deathrite Shaman is both black and green, and Judge's Familiar is both white and blue, so they each count towards Niv-Mizzet's X.[/a]

[hr]
[q]If I copy [c]Wildwood Scourge[/c] with X=5, does the copy also enter with five counters?[/q]

[a]That depends on how you copy it. If you make a token copy of a Wildwood Scourge that's already on the battlefield, for example with [c]Spitting Image[/c], then no. The copy's X is 0, so it enters as a sad 0/0 without any +1/+1 counters on it. However, if Wildwood Scourge is a spell on the stack and you're copying it with [c]Magus Lucea Kane[/c]'s ability, then the copied spell has the same X as the original, and the resulting token will enter with the same number of counters.[/a]

[hr]
[q]If [c]Maze of Ith[/c] gets an everything counter from [c]Omo, Queen of Vesuva[/c], can I tap it for green mana?[/q]

[a]Sure, as long as Omo sticks around. Omo's ability gives Maze of Ith every land type from Cave to Urza's and everything in between, including Forest. As a consequence of becoming a Forest, Maze of Ith gains the intrinsic ability to produce green mana. [/a]

[hr]
[q]Does [c]Choking Vines[/c] do anything against an attacker with trample such as [c]Giant Warthog[/c]?[/q]

[a]Not really. While the Warthog becomes blocked due to Choking Vines, there is no creature there that's blocking it when combat damage is assigned. As we've seen last week, this means that all its damage gets assigned to the thing it's attacking, which is exactly what would have happened if you hadn't cast Choking Vines.[/a]

[hr]
[q]Is putting [c]Bigger on the Inside[/c] on [c]Pili-Pala[/c] as ridiculous as I think it is?[/q]

[a]If you think that this gives you an arbitrary amount of mana and cascade triggers, you're exactly right! Because of Bigger on the Inside, you can tap Pili-Pala to create two mana and to give your next spell cascade. You can use those two mana to untap Pili-Pala for its own ability to make one mana, and now you're back to the same game state where you started except for an extra mana and an extra cascade ability on your next spell. Lather, rinse, repeat as needed.[/a]

[hr]
[q]Does [c]Insidious Roots[/c]'s ability get triggered by [c]Ojer Kaslem, Deepest Growth[/c] returning from the graveyard transformed? Or by [c=Mosswood Dreadknight // Dread Whispers]Mosswood Dreadknight[/c] being cast from my graveyard as an adventure? Or by [c]Enduring Vitality[/c] returning as an enchantment?  [/q]

[a]The answer is yes for all three scenarios. In each one, you have a creature card leaving your graveyard, and that's all that Insidious Roots cares about. It doesn't matter what zone the creature card goes to or whether it's still a creature by the time it gets there.[/a]

[hr]

And that's all the time we have for today. Thanks for reading, I hope you have a safe and happy Saint Patrick's Day, and please come back next week for more [b]Magic[/b] rules Q&A.

- Carsten Haese


----------------------

## Translate the tags for Card Name to their French versions

I already did this back in June 2024, I'm just going to do it again.
This time, I want to be bullet-proof: I'll have three different solutions to try to translate a card's name from English to French!

In [50]:
display(Markdown(latest_article_text[:1000]))

[cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.

As always, if you have questions for us, you can email them to us at [email]moko@cranialinsertion.com[/email], or tweet short questions to our X account @CranialTweet. One of our authors will reply to you, and your question might appear in a future article.

[hr]
[q]Let's say I control [c]Hardened Scales[/c] and I cast [c]Bounty of the Hunt[/c], assigning one counter to each of three target creatures. Because of Harde

In [51]:
import re
find_card_tag_name = re.compile(r"\[c\][\w ,']+\[/c\]")

card_tag_names = find_card_tag_name.findall(latest_article_text)

for i, card_tag_name in enumerate(card_tag_names):
    print(f"The {i+1}-th card tag found in the article was named « {card_tag_name} »")

The 1-th card tag found in the article was named « [c]Hardened Scales[/c] »
The 2-th card tag found in the article was named « [c]Bounty of the Hunt[/c] »
The 3-th card tag found in the article was named « [c]Spelunking[/c] »
The 4-th card tag found in the article was named « [c]Titania, Gaea Incarnate[/c] »
The 5-th card tag found in the article was named « [c]Ashaya, Soul of the Wild[/c] »
The 6-th card tag found in the article was named « [c]Cultivator Colossus[/c] »
The 7-th card tag found in the article was named « [c]Plaxcaster Frogling[/c] »
The 8-th card tag found in the article was named « [c]Murder[/c] »
The 9-th card tag found in the article was named « [c]Biophagus[/c] »
The 10-th card tag found in the article was named « [c]Nylea, God of the Hunt[/c] »
The 11-th card tag found in the article was named « [c]Displaced Dinosaurs[/c] »
The 12-th card tag found in the article was named « [c]Wild Dogs[/c] »
The 13-th card tag found in the article was named « [c]Winter, Cynical O

I need this `nest_asyncio` dependency, apparently... to use the Scrython module (<https://pypi.org/project/scrython/>).

In [24]:
import nest_asyncio
nest_asyncio.apply()
from pprint import pprint

### First solution: using mtgsdk
I wrote a first function to translate the name of an English card into French, using mtgsdk python module (<https://github.com/MagicTheGathering/mtg-sdk-python>) to access the great MTG API <https://docs.magicthegathering.io/>:

In [52]:
%%time
!pip install mtgsdk
import mtgsdk  # https://github.com/MagicTheGathering/mtg-sdk-python

def translate_cardname_mtgsdk(cardname="Thalia, Guardian of Thraben", lang='fr', language='French'):
    cards = mtgsdk.Card.where(name=cardname).all()

    cardname_fr = cardname
    for card in cards:
        if card.foreign_names:
            foreign_name_fr = [ fn for fn in card.foreign_names if fn['language'] == language ]
            # print(f"Looking using card.foreign_names in {language} language:")
            # pprint(foreign_name_fr)
            if len(foreign_name_fr) > 0:
                if foreign_name_fr[0]['name']:
                    cardname_fr = foreign_name_fr[0]['name']
        else:
            # print(f"Looking using card.set: {card.set}, and name")
            cards_fr = mtgsdk.Card.where(set=card.set, name=cardname, lang=lang).all()
            for card_fr in cards_fr:
                if card_fr.name:
                    cardname_fr = card_fr.name
        if cardname_fr and cardname_fr != cardname:
            return cardname_fr

# Demo:
cardname = "Thalia, Guardian of Thraben" # my favorite card lol lol
print(f"The card name in English is « {cardname} »")
cardname_fr = translate_cardname_mtgsdk(cardname)
print(f"The card name in French is « {cardname_fr} »")

The card name in English is « Thalia, Guardian of Thraben »
The card name in French is « Thalia, vigile de Thraben »
CPU times: user 73 ms, sys: 8.54 ms, total: 81.5 ms
Wall time: 4.29 s


I don't know how, but this mtgsdk solution hallucinates the name of Thalia and translate it wronlgy... well, it will be a backup solution, not the first one.

### Second solution: using Scrython
A second translation function, using scrython module (<https://pypi.org/project/scrython/>):

In [37]:
%%time
!pip install scrython
import scrython

def translate_cardname_scrython(cardname="Thalia, Guardian of Thraben", lang='fr'):
    card = scrython.cards.Named(fuzzy=cardname)
    #print(f"\n# Card name in English = « {card.name()} »")
    # get the card Object of the translated version of this card
    set_code = card.set_code()
    collector_number = card.collector_number()
    try:
        search_card_fr = scrython.Collector(code=set_code, collector_number=collector_number, lang=lang)
        cardname_fr = search_card_fr.printed_name()
        #print(f"Nom de la carte en Français = « {cardname_fr} »")
        return cardname_fr
    except scrython.ScryfallError:
        #print(f"Nom de la carte en Français = pas trouvé !!!")
        return cardname

# Demo:
cardname = "Thalia, Guardian of Thraben" # my favorite card lol lol
print(f"The card name in English is « {cardname} »")
cardname_fr = translate_cardname_scrython(cardname)
print(f"The card name in French is « {cardname_fr} »")

The card name in English is « Thalia, Guardian of Thraben »
The card name in French is « Thalia, gardienne de Thraben »
CPU times: user 57.2 ms, sys: 4.32 ms, total: 61.5 ms
Wall time: 3.06 s


### Third solution: using Magic-Ville.com

I think I have to try to write a function using Magic-Ville (<https://www.magic-ville.com/fr/>) webpage or API, as they are the quickest website to provide French names...
As I'm writing this, Tarkir: Dragonstorm has been pre-released, and both the mtgsdk and the Scrython approach fail on the newest cards... but they are translated on Magic-Ville

In [27]:
import urllib.parse

def url_encode_cardname(cardname):
  """URL encodes the given card name."""
  return urllib.parse.quote(cardname)

# Example usage (assuming 'CARDNAME' is defined somewhere):
cardname = "Thalia, Guardian of Thraben"
encoded_cardname = url_encode_cardname(cardname)
print(encoded_cardname)

cardname = "Ugin, Eye of the Storms"
encoded_cardname = url_encode_cardname(cardname)
print(encoded_cardname)

Thalia%2C%20Guardian%20of%20Thraben
Ugin%2C%20Eye%20of%20the%20Storms


In [28]:
!pip install requests chardet
import requests
import chardet

def search_magic_ville(cardname):
    encoded_cardname = url_encode_cardname(cardname)
    url = f"https://magic-ville.com/fr/resultats?color_search=1&spe_options=selected&type_search=1&costx=1&endx=1&forx=1&recherche_titre={encoded_cardname}&fra=1&eng=1&graph_aff=1&ordre=NAMEFRA"
    response = requests.get(url) #, headers={'content-encoding': 'UTF-8'})

    response.encoding = 'ISO-8859-1'
    # response.encoding = 'UTF-8'
    # Use `requests` to check the encoding first (which may already work)
    if response.encoding == 'ISO-8859-1':  # a common fallback encoding when auto-detect fails
        # Guess the encoding using chardet if `requests` doesn't work well
        result = chardet.detect(response.content)
        encoding = result['encoding']
        # print(f"Guessed Encoding: {encoding}")
    else:
        encoding = response.encoding
        # print(f"Encoding from response headers: {encoding}")
    response.encoding = encoding

    return response.text

from bs4 import BeautifulSoup

def get_main_div(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    main_div = soup.find(id='r_main1')
    return main_div

def extract_second_td_text(main_div):
    """
    Extracts the text content of the second <td> tag with class "S13"
    within a specific structure of the main_div.
    """
    try:
        table = main_div.find('table')
        tbody = table  #.find('tbody')
        tr = tbody.find('tr')
        td_tags = tr.find_all('td', class_='S13')
        if len(td_tags) >= 2:
            return td_tags[1].get_text(strip=True)
        #else:
        #    print("Error: Less than two <td> tags with class 'S13' found.")
    except AttributeError:
        #print("Error: Could not find the table, tbody, tr, or <td> tags.")
        pass
    return

In [29]:
%%time

def translate_cardname_magic_ville(cardname):
    result_page = search_magic_ville(cardname)
    main_div = get_main_div(result_page)
    cardname_fr = extract_second_td_text(main_div)
    title2 = None
    if cardname_fr is None or cardname_fr == cardname:
        soup = BeautifulSoup(result_page, 'html.parser')
        try:
            title = soup.find('title').get_text()
            #print(f"title = {title}")
        except Exception as exc:
            #print("Error:", exc)
            #print(f"soup = {soup}")
            magic_ville_card_id = str(soup).replace('<script>location="', '').replace('";</script>', '')
            magic_ville_URL = f"https://magic-ville.com/fr/{magic_ville_card_id}"

            response = requests.get(magic_ville_URL, headers={'content-encoding': 'utf-8'})

            response.encoding = 'ISO-8859-1'
            # response.encoding = 'UTF-8'
            # Use `requests` to check the encoding first (which may already work)
            if response.encoding == 'ISO-8859-1':  # a common fallback encoding when auto-detect fails
                # Guess the encoding using chardet if `requests` doesn't work well
                result = chardet.detect(response.content)
                encoding = result['encoding']
                #print(f"Guessed Encoding: {encoding}")
            else:
                encoding = response.encoding
                #print(f"Encoding from response headers: {encoding}")
            response.encoding = encoding

            response_text = response.text
            soup = BeautifulSoup(response_text, 'html.parser')
            #print(f"soup = {soup}")
            title = soup.find('title').get_text().replace(' - magic-ville.com', '')
            #print(f"title = {title}")
            title2 = soup.find('div', class_='S16').get_text(strip=True)
            #print(f"title2 = {title2}")

        cardname_fr = title
        #print(f"cardname_fr = {cardname_fr}")
        #print(f"title2 = {title2}")
    return cardname_fr  #, title2


# Demo:
#cardname = "Thalia, Guardian of Thraben" # my favorite card lol lol
#print(f"The card name in English is « {cardname} »")
#cardname_fr = translate_cardname_magic_ville(cardname)
#print(f"The card name in French is « {cardname_fr} »")

cardname = "Ugin, Eye of the Storms"  # a recent card
print(f"The card name in English is « {cardname} »")
cardname_fr = translate_cardname_magic_ville(cardname)
print(f"The card name in French is « {cardname_fr} »")

The card name in English is « Ugin, Eye of the Storms »
The card name in French is « Ugin, l'Œil des tempêtes »
CPU times: user 454 ms, sys: 22 ms, total: 476 ms
Wall time: 22.2 s


Of course, I'm unlucky enough to have picked the ONLY card in Tarkir: Dragonstorm with a weird non-LATIN1 symbol in its name...

> Ugin, l'Œil des tempêtes => Ugin, l'il des tempêtes!

OUPSIE. I tried and tried, nothing worked to get a sucessful handling of the encoding of the [Magic-Ville.com website](https://magic-ville.com/fr/carte.php?tdm409) <https://magic-ville.com/fr/carte.php?tdm409>.
Until ChatGPT gave a useful suggestion, now it works.

### A more robust solution

So we can hopefully be more robust, by having three translation functions instead of a single one:

In [59]:
from functools import lru_cache

@lru_cache(maxsize=None)
def translate_cardname(cardname="Thalia, guardian of Thraben", lang='fr', language='French'):
    try:
        cardname_fr = translate_cardname_scrython(cardname, lang=lang)
    except:
        cardname_fr = None

    if cardname_fr is None or cardname_fr == cardname:
        try:
            cardname_fr = translate_cardname_mtgsdk(cardname, lang=lang, language=language)
        except:
            cardname_fr = None

    if cardname_fr is None or cardname_fr == cardname:
        assert(lang == 'fr')
        assert(language == 'French')
        try:
            cardname_fr = translate_cardname_magic_ville(cardname)
        except:
            cardname_fr = None

    if cardname_fr is None or cardname_fr == cardname:
        print(f"Could not find a {language} name for « {cardname} »")
        return cardname
    return cardname_fr

In [31]:
%%time
# Demo
for i, cardname in enumerate([
        "Black Lotus",   # No French name! https://gatherer.wizards.com/Pages/Search/Default.aspx?name=+[Black Lotus]
        "Soulherder",    # Should work!
        "Goblin Matron", # Should work!
        "Thalia, Guardian of Thraben",  # Should work!
        "+2 Mace",       # didn't work, now it does, thanks to Magic-Ville !
        "Ugin, Eye of the Storms",  # Does not work  yet with first two methods, but should work using https://www.Magic-Ville.com/fr/
    ]):
        # Demo:
        print(f"\n# {i+1}-th card:\nThe card name in English is « {cardname} »")
        cardname_fr = translate_cardname(cardname)
        print(f"The card name in French is « {cardname_fr} »")


# 1-th card:
The card name in English is « Black Lotus »
The card name in French is « Lotus noir »

# 2-th card:
The card name in English is « Soulherder »
The card name in French is « Berger des âmes »

# 3-th card:
The card name in English is « Goblin Matron »
The card name in French is « Matrone gobeline »

# 4-th card:
The card name in English is « Thalia, Guardian of Thraben »
The card name in French is « Thalia, gardienne de Thraben »

# 5-th card:
The card name in English is « +2 Mace »
The card name in French is « Masse d'armes +2 »

# 6-th card:
The card name in English is « Ugin, Eye of the Storms »
The card name in French is « Ugin, l'Œil des tempêtes »
CPU times: user 1.81 s, sys: 120 ms, total: 1.93 s
Wall time: 1min 38s


As we see, some card requires to use MagicVille and the site is very slow, compae to the two othe solutions... It takes about 2 minutes to translate just SIX names! Crazy!
But well... **it works!**

I added a simple LRU cache in order to not take a long time to translate twice the same cardname. It should speed up in some case, and it doesn't cost uch.

In [32]:
%%time
# Demo
for i, cardname in enumerate([
        "+2 Mace",       # didn't work, now it does, thanks to Magic-Ville !
        "Ugin, Eye of the Storms",  # Does not work  yet with first two methods, but should work using https://www.Magic-Ville.com/fr/
    ]):
        # Demo:
        print(f"\n# {i+1}-th card:\nThe card name in English is « {cardname} »")
        cardname_fr = translate_cardname(cardname)
        print(f"The card name in French is « {cardname_fr} »")


# 1-th card:
The card name in English is « +2 Mace »
The card name in French is « Masse d'armes +2 »

# 2-th card:
The card name in English is « Ugin, Eye of the Storms »
The card name in French is « Ugin, l'Œil des tempêtes »
CPU times: user 106 µs, sys: 23 µs, total: 129 µs
Wall time: 103 µs


### Translate the cardname tags in the entire document

Now we're ready to try to translate the cardnames tags automatically:

In [53]:
display(Markdown(latest_article_text[1000:2000]))

ned Scales, this will result in two counters being put onto each target. Do I have to remove all six counters or just one counter per creature at the beginning of the cleanup step?[/q]

[a]I'm afraid you'll have to remove all six counters. Bounty of the Hunt tracks how many counters each creature received "this way", which means how many counters it received due to following the instructions of Bounty of the Hunt as modified by any applicable replacement effects. Since each target creature received two counters this way, you'll have to remove two counters from each of them.[/a]

[hr]
[q]If I control [c]Spelunking[/c] and [c]Titania, Gaea Incarnate[/c] enters, do the lands return from the graveyard tapped or untapped?[/q]

[a]The lands will enter untapped unless the land itself has a replacement effect that says it enters tapped, in which case you get a choice between tapped and untapped. Titania's effect instructing you to return the lands to the battlefield tapped is not a replacement

In [54]:
%%time

for i, card_tag_name in enumerate(card_tag_names):
    cardname = card_tag_name.replace('[c]', '').replace('[/c]', '')
    print(f"- The {i+1}-th card found is withing a tag « {card_tag_name} », named « {cardname} »...")
    cardname_fr = translate_cardname(cardname)
    if cardname != cardname_fr:
        print(f"  => elle semble traduite en « {cardname_fr} »")
        latest_article_text = latest_article_text.replace(
            f"{card_tag_name}",
            f"[c={cardname}]{cardname_fr}[/c]"
        )
    if i >= 1:
        break

display(Markdown(latest_article_text[1000:2000]))

- The 1-th card found is withing a tag « [c]Hardened Scales[/c] », named « Hardened Scales »...
  => elle semble traduite en « Écailles renforcées »
- The 2-th card found is withing a tag « [c]Bounty of the Hunt[/c] », named « Bounty of the Hunt »...
  => elle semble traduite en « Prime de chasse »


e target creatures. Because of Hardened Scales, this will result in two counters being put onto each target. Do I have to remove all six counters or just one counter per creature at the beginning of the cleanup step?[/q]

[a]I'm afraid you'll have to remove all six counters. Bounty of the Hunt tracks how many counters each creature received "this way", which means how many counters it received due to following the instructions of Bounty of the Hunt as modified by any applicable replacement effects. Since each target creature received two counters this way, you'll have to remove two counters from each of them.[/a]

[hr]
[q]If I control [c]Spelunking[/c] and [c]Titania, Gaea Incarnate[/c] enters, do the lands return from the graveyard tapped or untapped?[/q]

[a]The lands will enter untapped unless the land itself has a replacement effect that says it enters tapped, in which case you get a choice between tapped and untapped. Titania's effect instructing you to return the lands to the bat

CPU times: user 84.4 ms, sys: 13.8 ms, total: 98.2 ms
Wall time: 1.87 s


Now that it worked using a few cells, on a given article, let's write a function which does exactly this but on a given text, so we can add it in the translation pipeline.

In [60]:
import re

def translate_card_tag_names(latest_article_text, lang='fr', language='French'):
    find_card_tag_name = re.compile(r"\[c\][\w ,']+\[/c\]")

    card_tag_names = find_card_tag_name.findall(latest_article_text)

    for i, card_tag_name in enumerate(card_tag_names):
        cardname = card_tag_name.replace('[c]', '').replace('[/c]', '')
        print(f"The {i+1}-th card found is withing a tag « {card_tag_name} », named « {cardname} »...")
        cardname_fr = translate_cardname(cardname, lang=lang, language=language)
        if cardname != cardname_fr:
            print(f"  => it seems to be translated (to {language}) into « {cardname_fr} »")
            latest_article_text = latest_article_text.replace(
                f"{card_tag_name}",
                f"[c={cardname}]{cardname_fr}[/c]"
            )
    return latest_article_text

We can finally try to translate each cardname tags in the latest article:

In [61]:
%%time
latest_article_text = translate_card_tag_names(latest_article_text)
display(Markdown(latest_article_text))

The 1-th card found is withing a tag « [c]Spelunking[/c] », named « Spelunking »...
  => it seems to be translated (to French) into « Spéléologie »
The 2-th card found is withing a tag « [c]Titania, Gaea Incarnate[/c] », named « Titania, Gaea Incarnate »...
  => it seems to be translated (to French) into « Titania, Voix de Gaia »
The 3-th card found is withing a tag « [c]Ashaya, Soul of the Wild[/c] », named « Ashaya, Soul of the Wild »...
  => it seems to be translated (to French) into « Ashaya, âme de la nature »
The 4-th card found is withing a tag « [c]Cultivator Colossus[/c] », named « Cultivator Colossus »...
  => it seems to be translated (to French) into « Colosse cultivateur »
The 5-th card found is withing a tag « [c]Plaxcaster Frogling[/c] », named « Plaxcaster Frogling »...
  => it seems to be translated (to French) into « Grenouilleau lanceplax »
The 6-th card found is withing a tag « [c]Murder[/c] », named « Murder »...
  => it seems to be translated (to French) into « Ho

[cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.

As always, if you have questions for us, you can email them to us at [email]moko@cranialinsertion.com[/email], or tweet short questions to our X account @CranialTweet. One of our authors will reply to you, and your question might appear in a future article.

[hr]
[q]Let's say I control [c=Hardened Scales]Écailles renforcées[/c] and I cast [c=Bounty of the Hunt]Prime de chasse[/c], assigning one counter to each of three target creatures. Because of Hardened Scales, this will result in two counters being put onto each target. Do I have to remove all six counters or just one counter per creature at the beginning of the cleanup step?[/q]

[a]I'm afraid you'll have to remove all six counters. Bounty of the Hunt tracks how many counters each creature received "this way", which means how many counters it received due to following the instructions of Bounty of the Hunt as modified by any applicable replacement effects. Since each target creature received two counters this way, you'll have to remove two counters from each of them.[/a]

[hr]
[q]If I control [c=Spelunking]Spéléologie[/c] and [c=Titania, Gaea Incarnate]Titania, Voix de Gaia[/c] enters, do the lands return from the graveyard tapped or untapped?[/q]

[a]The lands will enter untapped unless the land itself has a replacement effect that says it enters tapped, in which case you get a choice between tapped and untapped. Titania's effect instructing you to return the lands to the battlefield tapped is not a replacement effect, it's just the effect of what the ability does. If the land doesn't have a replacement effect itself, then Spelunking's effect is the only replacement effect involved, and it replaces Titania's "tapped" with "untapped". If the land has a replacement effect along the lines of "this land enters tapped", then you choose the order in which to apply the land's replacement effect and Spelunking's effect, which results in a choice between tapped and untapped.[/a]

[hr]
[q]If I control [c=Ashaya, Soul of the Wild]Ashaya, âme de la nature[/c], can I put creatures onto the battlefield with [c=Cultivator Colossus]Colosse cultivateur[/c]'s ability because they're also lands?[/q]

[a]No, that doesn't work. Ashaya's ability only affects nontoken creatures that are on the battlefield. It does not affect the creature cards that are in your hand.[/a]

[hr]
[q]I control a [c=Plaxcaster Frogling]Grenouilleau lanceplax[/c] that still has a +1/+1 counter left on it and my opponent tries to [c=Murder]Homicide[/c] it. If I activate the Frogling's ability to give it shroud, does that make Murder fizzle?[/q]

[a]Absolutely. Plaxcaster Frogling can target itself with its ability, and the ability resolves first, giving the Frogling shroud. When Murder tries to resolve, it finds that its only target has become an illegal target, so it doesn't resolve and just goes to the graveyard.[/a]

[hr]
[q]If I use [c]Biophagus[/c]'s mana to cast [c=Nylea, God of the Hunt]Nyléa, déesse de la Chasse[/c] but I don't have enough devotion to green for Nylea to be a creature, does it still get the +1/+1 counter?[/q]

[a]Yup! Since Nylea is by default a creature and the ability that turns off its creature-ness only functions on the battlefield, you are in fact using Biophagus's mana to cast a creature spell, so Nylea gets the replacement effect "enters with an additional +1/+1 counter" stapled onto it. There is nothing stopping a noncreature permanent from entering with a +1/+1 counter, and there is no reason for a +1/+1 counter to fall off from a noncreature permanent. The counter will be placed on Nylea and stays there, it just won't have any effect until Nylea is a creature.[/a]

[hr]
[q]If [c=Displaced Dinosaurs]Dinosaures anachroniques[/c] makes a planeswalker creature and that planeswalker creature gets attacked, can it block the attacker?[/q]

[a]Sure, that's perfectly legal and probably beneficial even though it does not change the type or amount of harm that's coming to the planeswalker. Whether you block the attacker or not, it will deal its combat damage to the planeswalker creature, and in either case this will cause the planeswalker creature to lose loyalty counters (because it's a planeswalker) and to have damage marked on it (because it's a creature). However, as a blocking creature the planeswalker creature gets to deal combat damage back to the attacker, and as a 7/7 Dinosaur it deals a pretty significant amount of damage.[/a]

[hr]
[cleft=Wild Dogs]Hear the dogs howling out of key
To a hymn called "Faith and Misery"[/cleft][q]What does [c=Wild Dogs]Chiens sauvages[/c]'s triggered ability do in Two-Headed Giant?[/q]

[a]It does very little. One might think that a player on the team with the greater life total would gain control of the Dogs, but that's not how this works. The ability checks before it even goes on the stack whether an individual [i]player[/i] has more life than each other individual player. When an effect in Two-Headed Giant asks for a player's life total, the answer is that player's team's shared life total, so it will always get the same number for the two players on the same team. This means that the condition the ability looks for can never be true in Two-Headed Giant, so the ability never goes on the stack.[/a]

[hr]
[q]What are the card types that [c=Winter, Cynical Opportunist]Frimas, opportuniste cynique[/c] looks for?[/q]

[a]The term "card type" refers to the "main" card type of a card, which includes things like artifact, creature, land, and so on. On the type line there may be a supertype such as legendary or basic in front of the card type; those are not card types, so Winter isn't interested in those. After the dash you'll find subtypes such as Elf, Forest, or Equipment. Those aren't card types either, so Winter isn't interested in those, either.[/a]

[hr]
[q]Can [c=Winter, Cynical Opportunist]Frimas, opportuniste cynique[/c]'s ability put a battle's back face onto the battlefield? For example, can I put [c=Invasion of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c] onto the battelfield as [c=Zilortha, Apex of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c]?[/q]

[a]No. When an effect puts a double-faced card onto the battlefield without specifying a face, it is put onto the battlefield with its front face up.[/a]

[hr]
[q]Does [c=Garruk's Packleader]Chef de meute de Garruk[/c] see itself entering?[/q]

[a]A little bit yes, but mostly no. Every creature sees itself entering the battlefield, since enters abilities are checked after the event, when the creature is on the battlefield. However, Garruk's Packleader's ability only triggers whenever [i]another[/i] creature with power 3 or greater enters, so it doesn't trigger its own enters ability.[/a]

[hr]
[q]I'm in a game of Commander, my [c]Ms. Bumbleflower[/c]'s ability has triggered, but all my opponents have hexproof. Can I still give a creature a +1/+1 counter and flying?[/q]

[a]No. In order to put Ms. Bumbleflower's ability on the stack, you have to choose legal targets for each required target. The ability needs a target opponent and a target creature, and since you can't legally target an opponent, the ability is immediately removed from the stack.[/a]

[hr]
[q]I am casting [c=Reborn Hope]Espoir renaissant[/c] off of a [c=Counterpoint]Contrepoint[/c] that targeted a spell with mana value 2 or greater. Can Reborn Hope bring back this Counterpoint?[/q]

[a]No. You have to choose the target for Reborn Hope at the moment you cast it, and you're casting Reborn Hope while Counterpoint is resolving. At that moment, Counterpoint is on the stack, not in the graveyard, so you can't target it with Reborn Hope.[/a]

[hr]
[cright=Deathrite Shaman]I pledge allegiance to the underworld[/cright][q]Does [c]Niv-Mizzet, Guildpact[/c] count hybrid one-drops such as [c=Judge's Familiar]Familier du juge[/c] or [c=Deathrite Shaman]Shamane ritemort[/c] towards its X?[/q]

[a]Absolutely. The hybrid mana symbol has both of its colors, which means that the permanent is both of those colors. Deathrite Shaman is both black and green, and Judge's Familiar is both white and blue, so they each count towards Niv-Mizzet's X.[/a]

[hr]
[q]If I copy [c=Wildwood Scourge]Fléau de bois sauvage[/c] with X=5, does the copy also enter with five counters?[/q]

[a]That depends on how you copy it. If you make a token copy of a Wildwood Scourge that's already on the battlefield, for example with [c=Spitting Image]Portrait craché[/c], then no. The copy's X is 0, so it enters as a sad 0/0 without any +1/+1 counters on it. However, if Wildwood Scourge is a spell on the stack and you're copying it with [c]Magus Lucea Kane[/c]'s ability, then the copied spell has the same X as the original, and the resulting token will enter with the same number of counters.[/a]

[hr]
[q]If [c=Maze of Ith]Labyrinthe d'Ith[/c] gets an everything counter from [c=Omo, Queen of Vesuva]Omo, reine de Vésuva[/c], can I tap it for green mana?[/q]

[a]Sure, as long as Omo sticks around. Omo's ability gives Maze of Ith every land type from Cave to Urza's and everything in between, including Forest. As a consequence of becoming a Forest, Maze of Ith gains the intrinsic ability to produce green mana. [/a]

[hr]
[q]Does [c=Choking Vines]Vignes étrangleuses[/c] do anything against an attacker with trample such as [c=Giant Warthog]Phacochère géant[/c]?[/q]

[a]Not really. While the Warthog becomes blocked due to Choking Vines, there is no creature there that's blocking it when combat damage is assigned. As we've seen last week, this means that all its damage gets assigned to the thing it's attacking, which is exactly what would have happened if you hadn't cast Choking Vines.[/a]

[hr]
[q]Is putting [c=Bigger on the Inside]Plus grand à l'intérieur[/c] on [c]Pili-Pala[/c] as ridiculous as I think it is?[/q]

[a]If you think that this gives you an arbitrary amount of mana and cascade triggers, you're exactly right! Because of Bigger on the Inside, you can tap Pili-Pala to create two mana and to give your next spell cascade. You can use those two mana to untap Pili-Pala for its own ability to make one mana, and now you're back to the same game state where you started except for an extra mana and an extra cascade ability on your next spell. Lather, rinse, repeat as needed.[/a]

[hr]
[q]Does [c=Insidious Roots]Racines insidieuses[/c]'s ability get triggered by [c=Ojer Kaslem, Deepest Growth]Ojer Kaslem, Croissance des profondeurs // Temple de la culture[/c] returning from the graveyard transformed? Or by [c=Mosswood Dreadknight // Dread Whispers]Mosswood Dreadknight[/c] being cast from my graveyard as an adventure? Or by [c=Enduring Vitality]Vitalité perpétuelle[/c] returning as an enchantment?  [/q]

[a]The answer is yes for all three scenarios. In each one, you have a creature card leaving your graveyard, and that's all that Insidious Roots cares about. It doesn't matter what zone the creature card goes to or whether it's still a creature by the time it gets there.[/a]

[hr]

And that's all the time we have for today. Thanks for reading, I hope you have a safe and happy Saint Patrick's Day, and please come back next week for more [b]Magic[/b] rules Q&A.

- Carsten Haese


CPU times: user 1.81 s, sys: 153 ms, total: 1.96 s
Wall time: 1min 24s


------------------

## Now, let's translate the entire article!

I want to be able to obtain:

- the entire raw text,
- its splitting into paragraphs,
- their translations,

just from a given (or the latest) article ID.

In [63]:
from tqdm.notebook import tqdm

In [65]:
def get_and_translate_article_text(article_id=None, translate=False):

    if article_id is None:
        try:
            article_id = get_latest_article_id()
        except:
            article_id = 4380  # etc, see https://www.cranial-insertion.com/archive for the entire list

    latest_article_text, latest_article_title, latest_article_date, soup = get_article_text(article_id)

    article_content = translate_card_tag_names(latest_article_text)

    article_paragraphs = article_content.split("\n\n")
    translated_paragraphs = []

    for i, paragraph in tqdm(enumerate(article_paragraphs)):
        print(f"\n\n{i+1}-th / {len(article_paragraphs)} English paragraph (of length {len(paragraph)} letters and {len(paragraph.split(' '))} words)")
        print_paragraph(paragraph)

        if translate:
            translated_paragraph = en2fr(paragraph)
            print(f"Translated {i}-th paragraph in French (of length {len(translated_paragraph)} letters and {len(translated_paragraph.split(' '))} words)")
            print_paragraph(translated_paragraph)
            translated_paragraphs.append(translated_paragraph)

    return article_id, article_content, latest_article_title, latest_article_date, article_paragraphs, translated_paragraphs

In [88]:
%%time
# uncomment/comment the preferred option
translate = True
translate = False

article_id, latest_article_text, latest_article_title, latest_article_date, article_paragraphs, translated_paragraphs = get_and_translate_article_text(article_id, translate=translate)

Reading the article at URL https://www.cranial-insertion.com/staff/articles/4371/edit ...
Using French language and my editor cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !
The 1-th card found is withing a tag « [c]Hardened Scales[/c] », named « Hardened Scales »...
  => it seems to be translated (to French) into « Écailles renforcées »
The 2-th card found is withing a tag « [c]Bounty of the Hunt[/c] », named « Bounty of the Hunt »...
  => it seems to be translated (to French) into « Prime de chasse »
The 3-th card found is withing a tag « [c]Spelunking[/c] », named « Spelunking »...
  => it seems to be translated (to French) into « Spéléologie »
The 4-th card found is withing a tag « [c]Titania, Gaea Incarnate[/c] », named « Titania, Gaea Incarnate »...
  => it seems to be translated (to French) into « Titania, Voix de Gaia »
The 5-th card found is withing a tag « [c]Ashaya, Soul of the Wild[/c] », named « Ashaya, Soul o

0it [00:00, ?it/s]



0-th English paragraph (of length 577 letters and 94 words)


> [cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.



1-th English paragraph (of length 257 letters and 42 words)


...



39-th English paragraph (of length 180 letters and 34 words)


> And that's all the time we have for today. Thanks for reading, I hope you have a safe and happy Saint Patrick's Day, and please come back next week for more [b]Magic[/b] rules Q&A.



40-th English paragraph (of length 16 letters and 3 words)


> - Carsten Haese


CPU times: user 173 ms, sys: 16.1 ms, total: 189 ms
Wall time: 682 ms


When running on the GPU, the first try gave me a total running time of 40 seconds to generate these 57 first draft of translation, I think that's a great performance!

In [89]:
print("- The article ID is:", article_id)
latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"

print("- The latest article is here:", latest_article_link)
print("- It was published on:", latest_article_date)

- The latest article is here: https://www.cranial-insertion.com/article/4371
- It was published on: 03/17/2025
- The article ID is: 4371


In [90]:
print("- Its title in English is:")
print_paragraph(latest_article_title)
latest_article_title_french = en2fr(latest_article_title)
print("- A suggestion of a title in French is:")
print_paragraph(latest_article_title_french)

- Its title in English is:


> A Very Green Day

- A suggestion of a title in French is:


> Une journée très verte

Let's now generate the content to write to the Google doc:

Google doc does **not** support Markdown markup, of course (it's more a clone of Microsoft Word), so the markup I use below is not rendered correctly by default. There is a hack below for that (for titles and lists, nothing more).

In [85]:
def us_date_to_fr_date(us_date):
    month = us_date[0:2]
    day = us_date[3:5]
    year = us_date[6:]
    return f"{day}/{month}/{year}"

us_date_to_fr_date('04/17/2025')

'17/04/2025'

In [91]:
def generate_content_to_write(article_id, article_paragraphs, translated_paragraphs, latest_article_title, latest_article_date, translate=False):

    latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"

    print("- Its title in English is:")
    print_paragraph(latest_article_title)
    latest_article_title_french = en2fr(latest_article_title)
    print("- A suggestion of a title in French is:")
    print_paragraph(latest_article_title_french)

    latest_article_date_fr = us_date_to_fr_date(latest_article_date)

    entire_content_to_write = f"""# TITLE: « {latest_article_title} »

Une suggestion automatique de traduction de ce titre est :

## TITRE : « {latest_article_title_french} »

- Cet article vient de cette page web : {latest_article_link}
- Il a été ou sera publié le : {latest_article_date_fr}

--------------------------------------------------------
    """

    for i in range(len(article_paragraphs)):
        paragraph = article_paragraphs[i]
        #print(f"\n\n{i}-th English paragraph: ")

        if translate:
            paragraph = paragraph.replace("\n", "\n    ")
            entire_content_to_write += f"\n\n    {paragraph}"
            translated_paragraph = translated_paragraphs[i]
            #print(f"Translated {i}-th paragraph in French: ")
            entire_content_to_write += f"\n\n{translated_paragraph}"
        else:
            entire_content_to_write += f"\n\n{paragraph}"

    return entire_content_to_write

We are now ready to us the previous functions:

In [92]:
%%time
translate = False

article_id = 4371  # faster than get_latest_article_id()

article_id, latest_article_text, latest_article_title, latest_article_date, article_paragraphs, translated_paragraphs = get_and_translate_article_text(article_id, translate=translate)

entire_content_to_write = generate_content_to_write(article_id, article_paragraphs, translated_paragraphs, latest_article_title, latest_article_date, translate=translate)

print("The entire content to write is:")
print_paragraph(entire_content_to_write)

Reading the article at URL https://www.cranial-insertion.com/staff/articles/4380/edit ...
Using French language and my editor cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !
The 1-th card found is withing a tag « [c]Armament Dragon[/c] », named « Armament Dragon »...
Could not find a French name for « Armament Dragon »
The 2-th card found is withing a tag « [c]Sarkhan, Dragon Ascendant[/c] », named « Sarkhan, Dragon Ascendant »...
Could not find a French name for « Sarkhan, Dragon Ascendant »
The 3-th card found is withing a tag « [c]Osseous Exhale[/c] », named « Osseous Exhale »...
Could not find a French name for « Osseous Exhale »
The 4-th card found is withing a tag « [c]Poised Practitioner[/c] », named « Poised Practitioner »...
Could not find a French name for « Poised Practitioner »
The 5-th card found is withing a tag « [c]Taigam, Master Opportunist[/c] », named « Taigam, Master Opportunist »...
  => it seems to be

0it [00:00, ?it/s]



0-th English paragraph (of length 409 letters and 62 words)


> [cright=Dragonstorm Forecaster]Today's forecast:
Cloudy with a chance of Dragonstorms[/cright]Greetings and welcome back to another issue of Cranial Insertion! The prereleases for [i]Tarkir: Dragonstorm[/i] are in the books and the set is heading into stores this weekend, so it's time for our first look at the fearsome Dragons -- and I suppose some other creatures, too -- and the new mechanics in this set.



1-th English paragraph (of length 359 letters and 56 words)


...

- A suggestion of a title in French is:


> Nuageux avec une chance de Dragonstorms

The entire content to write is:


> # TITLE: « Cloudy With a Chance of Dragonstorms »

Une suggestion automatique de traduction de ce titre est :

## TITRE : « Nuageux avec une chance de Dragonstorms »

- Cet article vient de cette page web : https://www.cranial-insertion.com/article/4380
- Il a été ou sera publié le : 07/04/2025

--------------------------------------------------------
    

[cright=Dragonstorm Forecaster]Today's forecast:
Cloudy with a chance of Dragonstorms[/cright]Greetings and welcome back to another issue of Cranial Insertion! The prereleases for [i]Tarkir: Dragonstorm[/i] are in the books and the set is heading into stores this weekend, so it's time for our first look at the fearsome Dragons -- and I suppose some other creatures, too -- and the new mechanics in this set.

As always, if you have questions for us, whether they're about [i]Tarkir: Dragonstorm[/i] or [b]Magic[/b] cards from any other set, you can send your questions by email to [email]moko@cranialinsertion.com[/email], or send short questions on X to @CranialTweet. One of our authors will send you an answer, and your question may even appear in a future article.

And now, without further ado, let us behold some questions about Dragons!

[hr]
[q]What counts as a Dragon for [c=Caustic Exhale]behold a Dragon[/c]?[/q]

[a]Something is a Dragon if (and only if) it has the creature type Dragon on its type line, such as [c]Armament Dragon[/c] just to name one of many examples. Having artwork that looks like a Dragon doesn't count -- sorry, [c=Ugin, Eye of the Storms]Ugin[/c] -- and just having the word "Dragon" in the card name doesn't count, either -- sorry, [c]Sarkhan, Dragon Ascendant[/c].[/a]

[hr]
[q]If I choose a Dragon I control to behold for [c]Osseous Exhale[/c] and the Dragon gets destroyed in response, do I still gain 2 life?[/q]

[a]Sure. Osseous Exhale only checks whether a Dragon was beheld at the time it was cast. It doesn't matter if the Dragon is still around at the time Osseous Exhale resolves, you gain 2 life either way.[/a]

[hr]
[q]If [c]Poised Practitioner[/c] is the second spell I cast in a turn, does it get a +1/+1 counter?[/q]

[a]No. Poised Practitioner needs to be on the battlefield at the time you cast the second spell in a turn for its ability to trigger. If it's the second spell you're casting, it's on the stack at the time you're casting it, so its ability doesn't trigger.[/a]

[hr]
[q]If I control [c=Taigam, Master Opportunist]Taigam, maître opportuniste[/c] and cast [c]Channeled Dragonfire[/c] as my second spell for the turn, do both the original and the copy resolve before Taigam suspends the original?[/q]

[a]No. Taigam's ability goes on the stack above Channeled Dragonfire and resolves first. It makes a copy of Channeled Dragonfire, and then it exiles Channeled Dragonfire with four time counters on it. The copy resolves now-ish, and the suspended one will get a chance to resolve when its last time counter goes off, but the original Channeled Dragonfire won't resolve.[/a]

[hr]
[q]I control [c=Taigam, Master Opportunist]Taigam, maître opportuniste[/c] and the second spell I cast is the Omen spell [c=Stormshriek Feral // Flush Out]Flush Out[/c]. What happens?[/q]

[a]Taigam's ability makes a copy of the spell, and it sees the copiable characteristics of Flush Out, so it puts a copy of Flush Out on the stack. Then it suspends the entire card, Dragon and Omen, with four time counters on it. The Flush Out copy resolves and does its discard+draw thing and then you shuffle it into the library, which means you simply shuffle your library because the copy is not represented by a physical card. Eventually, when the last time counter comes off of the suspended card, you get to cast it for free and you may choose to cast either the Dragon side or the Omen side of it.[/a]

[hr]
[q]When do I have to announce the target for [c=Glacial Dragonhunt]Chasse au dragon glaciale[/c]?[/q]

[a]Glacial Dragonhunt as a spell doesn't have a target, so you don't announce a target at the time you cast it. If you choose to discard a card during its resolution and that card turns out to be a nonland card, this triggers a reflexive triggered ability that has a target, and you announce its target when you put that ability on the stack.[/a]

[hr]
[q]Does [c=Bloodletter of Aclazotz]Gravesang d'Aclazotz[/c] double the value of X for [c=Kotis, the Fangkeeper]Kotis, le Gardecroc[/c]'s triggered ability?[/q]

[a]I'm afraid not. Bloodletter of Aclazotz causes your opponent to lose twice the amount of life due the damage that Kotis deals to them, but the X in Kotis's ability is the amount of damage it dealt, not the amount of life that was lost as a result of the damage.[/a]

[hr]
[cleft=Breaching Dragonstorm|TDM|101]When the sky looks like that, RUN![/cleft][q]Speaking of [c=Kotis, the Fangkeeper]Kotis, le Gardecroc[/c], what's the time frame for casting the exiled cards?[/q]

[a]Since the ability doesn't specify a duration for the "you may cast" effect, it means [i]right now[/i], during the resolution of Kotis's triggered ability. Any cards that you choose not to cast at that time, or that you can't cast because they lack targets or other prerequisites to cast, simply stay in exile indefinitely and uselessly.[/a]

[hr]
[q]In Two-Headed Giant, does the last part of [c=Betor, Kin to All]Bétor, parent de tous[/c]'s ability just insta-kill my opposing team?[/q]

[a]As long as the "toughness 40 or greater" condition is met, sure. Both players on your opponents' team lose half their life rounded up at the same time, and the game sees their shared life total for each player's life total. For example, if the team's life total is 30, each opponent loses 15 life and their team ends up at 0 life. However, note that Betor only counts the total toughness of the creatures that you control individually. It does not consider the creatures your teammate controls.[/a]

[hr]
[q]If I control [c=The Sibsig Ceremony]La Cérémonie sibsig[/c] and I cast some creature, can I sacrifice it to give [c=Unburied Earthcarver]Creuseterre déterré[/c] a counter in response to the Ceremony's trigger and still get a Zombie Druid token? [/q]

[a]Yes, you can do that. The Ceremony's triggered ability will try and fail to destroy the creature, but the creation of the Zombie Druid token is not contingent on this destruction, so you still get the token.[/a]

[hr]
[q]Can I sacrifice a creature token to [c=Sidisi, Regent of the Mire]Sidisi, régente de la fondrière[/c]'s activated ability?[/q]

[a]Sure, as long as you can target a creature card with mana value 1 in your graveyard. To activate Sidisi's ability, you first choose some number for X; then you choose an appropriate target, which is a creature card with mana value X+1 in your graveyard; finally you pay the cost, which involves sacrificing a creature with mana value X other than Sidisi. X=0 is a legal choice for the ability, and a creature token has a mana value of 0 (usually, unless it's a copy of something that has a nonzero mana value). [/a]

[hr]
[q]After I draw cards with [c=Narset, Jeskai Waymaster]Narset, maîtresse de la Voie jeskaï[/c]'s ability, can I cast any of the cards I drew?[/q]

[a]If they're instants or have flash, sure. Narset's ability triggers and resolves in your end step, and players get priority in the end step, so you get the opportunity to cast spells from among the cards you drew.[/a]

[hr]
[q]For chapter III of [c=Awaken the Honored Dead]Éveiller les morts honorés[/c], when do I choose the target? Can I target the card I discarded?[/q]

[a]Choosing to discard a card triggers a reflexive ability that requires a target, so you only choose a target after you discard. This means that you could target and bring back the card you just discarded, provided that it's a creature or land card, but this only benefits you if some triggered ability notices the card entering or leaving the graveyard. Otherwise, you achieve the same end result of the card being in your hand by simply choosing not to discard it in the first place.[/a]

[hr]
[q]Can I target [c=Zurgo's Vanguard]Avant-garde de Zurgo[/c] with [c=Smile at Death]Alesha, Celle-qui-sourit-devant-la-mort[/c]'s triggered ability?[/q]

[a]That depends on how many creatures you control. The ability that defines Zurgo's Vanguard's power is a characteristic-defining ability that functions at all times and in all zones, including in your graveyard, so Zurgo's Vanguard's power is only small enough to be a legal target for the ability if you control at most two creatures.[/a]

[hr]
[cright=Call the Spirit Dragons]Who you gonna call?[/cright][q]Does [c=Call the Spirit Dragons]Appeler les dragons-esprits[/c]'s ability go on the stack even if I don't control a Dragon? In other words, could I respond to the trigger by casting a Dragon with flash and give it a +1/+1 counter with the ability?[/q]

[a]Sure, that works. Call the Spirit Dragons's ability does not have an intervening-if clause that gets checked at the time the ability triggers. The ability goes on the stack regardless of how many Dragons you control, so you can make more Dragons in response to the ability.[/a]

[hr]
[q]If [c=Effortless Master]Maître naturel[/c] is the second spell I cast in a turn, does it get the +1/+1 counters?[/q]

[a]Absolutely. As Effortless Master resolves, the replacement effect checks how many spells you've cast this turn, and you have cast two or more spells this turn, so the condition for Effortless Master to get counters is satisfied.[/a]

[hr]
[q]If [c=New Way Forward]Nouvelle voie à suivre[/c] deals an amount of damage that's lethal to my opponent but also causes me to draw more cards than I have left in my library, what happens?[/q]

[a]The game ends in a draw. The delayed triggered ability from New Way Forward's prevention effect deals damage to your opponent and causes you to draw a bunch of cards, and then state-based actions are checked. State-based actions see that your opponent is at 0 or less life and that you've attempted to draw from an empty library, so both of you lose the game at the same time and the game is a draw.[/a]

[hr]
[q]If I control [c=Hollowmurk Siege]Siège de Mornecreux[/c] with Sultai chosen, do I draw a card when [c=Host of the Hereafter]Hôte de l'après[/c] enters with counters on it?[/q]

[a]Yup! An ability that triggers when a counter is put on a permanent also triggers when a permanent enters with a counter on it. Also note that the ability would ordinarily trigger twice because Host of the Hereafter enters with two counters, but the ability is limited to triggering once each turn.[/a]

[hr]
[q]If [c=Mardu Siegebreaker]Brisesiège marduen[/c]'s ability triggers twice due to [c=Panharmonicon]Panharmonicus[/c] for example, what happens when its attack trigger resolves?[/q]

[a]Thanks to Panharmonicon, there are two cards that fit the description of "the exiled card", and the ability simply performs its actions on both of those cards, so you'll make tapped and attacking token copies of both cards for each opponent.[/a]

[hr]
[q]What happens if [c=Mardu Siegebreaker]Brisesiège marduen[/c]'s ability exiles a [c=Cubwarden]mutate pile[/c]?[/q]

[a]You'll get a very similar situation as in the previous question. Mardu Siegebreaker exiles one creature, but it turns into several individual cards in exile. All of those cards are "the exiled card", so you'll make token copies of each of those cards for each opponent.[/a]

[hr]

And that's all the time we have for today's episode. Thanks for reading, and until next time, watch out for stormy weather!

- Carsten Haese


CPU times: user 14.6 s, sys: 862 ms, total: 15.5 s
Wall time: 16min 38s


---------

## Great! Now, let's upload this text automatically (to a new tab?) in our Google doc!

First, I'll use a personnal Google doc, to test things out.

In [93]:
print_paragraph(entire_content_to_write[:1000])

> # TITLE: « Cloudy With a Chance of Dragonstorms »

Une suggestion automatique de traduction de ce titre est :

## TITRE : « Nuageux avec une chance de Dragonstorms »

- Cet article vient de cette page web : https://www.cranial-insertion.com/article/4380
- Il a été ou sera publié le : 07/04/2025

--------------------------------------------------------
    

[cright=Dragonstorm Forecaster]Today's forecast:
Cloudy with a chance of Dragonstorms[/cright]Greetings and welcome back to another issue of Cranial Insertion! The prereleases for [i]Tarkir: Dragonstorm[/i] are in the books and the set is heading into stores this weekend, so it's time for our first look at the fearsome Dragons -- and I suppose some other creatures, too -- and the new mechanics in this set.

As always, if you have questions for us, whether they're about [i]Tarkir: Dragonstorm[/i] or [b]Magic[/b] cards from any other set, you can send your questions by email to [email]moko@cranialinsertion.com[/email], or send short q

In [94]:
our_google_doc_url = "https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing"
print("The Google doc URL is:")
print(our_google_doc_url)

our_google_doc_id = our_google_doc_url.split("/")[-2]
#from google.colab import userdata
#our_google_doc_id = userdata('GOOGLE_DOC_ID')
print("The Google doc ID is:")
print(our_google_doc_id)

The Google doc URL is:
https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing
The Google doc ID is:
1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo


Let's try to write a text in a new tab of this Google doc.
It requires this code:

In [95]:
from googleapiclient.discovery import build
from google.colab import auth

auth.authenticate_user()

In [96]:
default_content_to_write_to_this_google_doc = """
# Title of level 1
## Title of level 2
### Title of level 3

*Hello, world!*
**WHO awesome!**

> Sent from a Google colab notebook running Python! Code by @Naereen

Using <https://colab.research.google.com/> for free (but maybe soon for Pro).
See [this page for more information](https://colab.research.google.com/signup).
"""

Thanks to Pragati Gunai's small GitHub project, [see its notebook](https://github.com/PragatiGunai1097/google-docs-markdown-formatter/blob/main/MarkdownMeeting_ToGoogleDoc_.ipynb):

In [97]:
# Step 3: Parse markdown to structure content
def markdown_to_requests(md_content, index=0):
    lines = md_content.split("\n")
    requests = []
    index = index + 1

    for line in lines:
        if line.startswith("# "):  # Heading 1
            requests.append({"insertText": {"location": {"index": index}, "text": line[2:] + "\n"}})
            index += len(line[2:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[2:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_1",  # Use HEADING_1 for main heading
                    },
                    "fields": "namedStyleType",
                }
            })
        elif line.startswith("## "):  # Heading 2
            requests.append({"insertText": {"location": {"index": index}, "text": line[3:] + "\n"}})
            index += len(line[3:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[3:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_2",  # Use HEADING_2 for main heading
                    },
                    "fields": "namedStyleType",
                }
            })
        elif line.startswith("### "):  # Heading 3
            requests.append({"insertText": {"location": {"index": index}, "text": line[4:] + "\n"}})
            index += len(line[4:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[4:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_3",  # Use HEADING_3 for main heading
                    },
                    "fields": "namedStyleType",
                }
            })
        elif line.startswith("#### "):  # Heading 4
            requests.append({"insertText": {"location": {"index": index}, "text": line[5:] + "\n"}})
            index += len(line[5:]) + 1
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(line[5:]) - 1, "endIndex": index - 1},
                    "paragraphStyle": {
                        "namedStyleType": "HEADING_4",  # Use HEADING_4 for main heading
                    },
                    "fields": "namedStyleType",
                }
            })
        elif line.startswith("- [ ]"):  # Checkboxes
            requests.append({"insertText": {"location": {"index": index}, "text": "☐ " + line[6:] + "\n"}})
            index += len("☐ " + line[6:]) + 1
        elif line.startswith("- "):  # Bullet points
            requests.append({"insertText": {"location": {"index": index}, "text": "• " + line[2:] + "\n"}})
            index += len("• " + line[2:]) + 1
        elif line.strip():  # Normal text
            requests.append({"insertText": {"location": {"index": index}, "text": line + "\n"}})
            index += len(line) + 1
        else:  # Blank lines
            requests.append({"insertText": {"location": {"index": index}, "text": "\n"}})
            index += 1

    return requests

We can use a simple request, or the more-complex request from above:

In [98]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/documents']
# The ID of the document to insert text into.
DOCUMENT_ID = our_google_doc_id

def write_content_to_google_doc(content_to_write=default_content_to_write_to_this_google_doc, markdown=False):
    service = build('docs', 'v1', credentials=None)

    # Create a new tab/section in the document.
    requests = [
        {'insertText':
            {
                'location': {'index': 1},
                'text':
                    content_to_write
            }
        },
    ]

    # Convert the markdown content into Google Docs API requests
    if markdown:
        requests = markdown_to_requests(content_to_write)

    result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()
    print(result)

    print(f"The Google doc at this URL has been modified with the desired content of length {len(content_to_write)}:")
    print(our_google_doc_url)

I wanted to be able to create a new tab, but apparently this can't be done using the current version of Google Docs API.

In [ ]:
# prompt: Create a new tab in this Google document, using the Google Docs API requests
def create_new_tab_in_google_doc(content, new_tab_name="New tab", markdown=False, document_id=DOCUMENT_ID):
    """Creates a new tab/section in a Google Doc and inserts content.

    Args:
        content: The content to insert into the new tab.
        document_id: The ID of the Google Doc.
    """
    try:
        service = build('docs', 'v1', credentials=None)
        # Find the current document's last index
        document = service.documents().get(documentId=document_id).execute()
        last_index = document['body']['content'][-1]['endIndex']
        print(f"last_index = {last_index} ...")
        last_index = 1
        # Create a new section at the end
        requests = [
            {
                'insertSectionBreak': {
                    'location': {'index': last_index},
                    'sectionType': 'NEXT_PAGE'  # Or SECTION_BREAK
                }
            }
        ]

        # Set the name of the section
        requests += [
            {
                "updateSectionStyle": {
                    "sectionStyle": {
                        "sectionType": "NEXT_PAGE",
                        #"info": {
                        #    "sectionId": new_tab_name, # new_tab_name is used as sectionId
                        #    "displayName": new_tab_name
                        #}
                    },
                    "range": {
                        "startIndex": last_index + 1,
                        "endIndex": last_index + 2,
                    },
                    #"fields": "info.displayName,info.sectionId"
                    "fields": "*"
                }
            }
        ]

        # Write the text as a basic index in the current tab
        if not markdown:
            requests += [
                {'insertText': {
                    'location': {'index': last_index + 1},
                    'text': content
                }}
            ]
        else:
            # Convert the markdown content into Google Docs API requests
            requests += markdown_to_requests(content, index=last_index+2)

        # Execute the requests
        result = service.documents().batchUpdate(
            documentId=document_id, body={'requests': requests}).execute()
        print(f"Successfully created a new tab in the Google Doc: {result}")
    except Exception as e:
        print(f"An error occurred: {e}")
        raise e

We can now test it! It should support basic formatting using Markdown.

In [99]:
%%time
write_content_to_google_doc(default_content_to_write_to_this_google_doc, markdown=True)

{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'writeControl': {'requiredRevisionId': 'ALBJ4LvX9CxMm8NUwCM6XWLsRt0Qg_nGLf6bAAZ3ikWW6GIhOLna_K9o_QAtXpkTvX9f1q5ZEKs1q1eZZ3Yjkw'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'}
The Google doc at this URL has been modified with the desired content of length 324:
https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing
CPU times: user 226 ms, sys: 52.2 ms, total: 278 ms
Wall time: 1.59 s


I tried for a long time different requests to add a new tab, but I'm pretty sure it's not supported by the API. See <https://googleapis.github.io/google-api-python-client/docs/dyn/docs_v1.documents.html> for instance.

In [ ]:
%%time
if False:
    # Example usage (replace with your actual content)
    new_tab_content = "This is the content of the new tab."

    create_new_tab_in_google_doc(new_tab_content, new_tab_name="Nouvel onglet")

last_index = 2 ...
An error occurred: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo:batchUpdate?alt=json returned "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.". Details: "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.">


HttpError: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo:batchUpdate?alt=json returned "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.". Details: "Invalid requests[1].updateSectionStyle: A value for top margin must be specified in order to update it.">

Now, if adding this first line worked, let's update the Google doc with all the paragraphs and their draft of translation.

In [100]:
def get_article_translate_it_then_upload_to_google_doc(article_id=None, translate=False):
    if article_id is None:
        article_id = get_latest_article_id()
    print(f"- The latest article ID is: {article_id}... Downloading it, then translating it...")

    article_id, latest_article_text, latest_article_title, latest_article_date, article_paragraphs, translated_paragraphs = get_and_translate_article_text(article_id, translate=translate)

    print("- Generating the Markdown content to write to Google doc:")
    entire_content_to_write = generate_content_to_write(article_id, article_paragraphs, translated_paragraphs, latest_article_title, latest_article_date, translate=translate)

    print("- The entire content to write is:")
    print_paragraph(entire_content_to_write)

    write_content_to_google_doc(entire_content_to_write, markdown=True)

    return latest_article_title, latest_article_date

In [ ]:
%%time
if False:
    get_article_translate_it_then_upload_to_google_doc(article_id=get_latest_article_id(), translate=False)

- The latest article ID is: 4376... Downloading it, then translating it...
Reading the article at URL https://www.cranial-insertion.com/staff/articles/4376/edit ...
Using French language and my admin cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !


0it [00:00, ?it/s]



0-th English paragraph (of length 93 letters and 12 words)


> [cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]



1-th English paragraph (of length 226 letters and 38 words)


...



57-th English paragraph (of length 62 letters and 10 words)


> - Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

- Generating the Markdown content to write to Google doc:
The entire content to write is:


> 
    # TITLE: « Cranial Insertion: All in Jeopardy (Can you answer in the form of a question?) »

    - Une suggestion pour la traduction de ce titre en français est :
    TITRE : « Cranial Insertion: All in Jeopardy (Pouvez-vous répondre sous la forme d'une question?) »

    - Le dernier article vient de cette page web : https://www.cranial-insertion.com/article/4376
    - Il a été publié le : Mon, 31 Mar 2025 00:00:00 EDT

    -----------------------------------------------------
    

[cright=Living Death]"Alex, the answer is "What would be an
example of an oximoron?"[/cright]

Today we have a collection of questions found on the IRC Chatroom #magicjudges-rules and the Facebook group Ask the Judge - [b]Magic[/b]: The Gathering Rules and Policy Questions. Feel free to join us in either or both groups.

If you have any [b]Magic[/b] questions burning at the back of your brain, you can send them to us. We may even use them in a future article. If you have a short question, you can send it to us via our Twitter account at [url=https://twitter.com/CranialTweet]@CranialTweet[/url], and you can send us longer questions at [email]moko@cranialinsertion.com[/email].

[hr]
[Q] I have two [c]Squire[/c]s on the battlefield and I have [c]Custodi Soulbinders[/c] and four copies of [c]Bear Cub[/c] in my graveyard. If I cast [c]Living Death[/c], how many +1/+1 counters will Soulbinders enter with?[/Q]

[A] Custodi Soulbinders will enter with zero +1/+1 counters. Souldbinders and the Bears will enter simultaneously. Soulbinders' first ability is a replacement effect that checks to see how many creatures are on the battlefield just before it enters. At that point there are no creatures on the board, so it gets no counters as it enters.[/A]

[hr]
[Q] My commander is [c]Grimgrin, Corpse-Born[/c]. Grimgrin was [c]Murder[/c]ed last turn and I choose to move him to the command zone. I control [c]Rooftop Storm[/c]. If I try to cast Grimgrin from the command zone, will he cost [m]2[/m] or will he be free?[/Q]

[A] He will cost [m]2[/m]. 
Commander "tax" is an additional cost that must be paid in addition to the normal cost for casting your commander. Rooftop Storm gives you the option of replacing the normal cost of casting your Zombies with [m]0[/m]. This has no effect on additional costs.  [/A]

[hr]
[Q] My opponent exiled [c]Wheel of Fate[/c] with [c]Reckless Impulse[/c]. She now claims she can cast Wheel of Fate because it's in exile. Is that true?[/Q]

[A] Your opponent is wrong. The part of suspend that lets you cast a card with no mana cost is casting it without paying its mana cost, not that it's casting it from exile.[/A]

[hr]
[Q] I control four [c]Bear Cub[/c]s, [c]Agrus Kos, Eternal Soldier[/c], and [c]Thundering Raiju[/c]. None of my creatures are currently modified. I attack with the Raiju and have its attack trigger target Agrus. When Agrus's ability triggers because he became targeted, I pay the cost. After all these triggers resolve, how much damage will Raiju deal to each opponent?[/Q]

[A] The Answer is between 15-19 damage, depending on when you let the trigger targeting Raiju resolve.

When Agrus's trigger resolves, it creates five copies of Raiju's trigger, one for each Bear and one for Raiju, making a total of six Raiju triggers. The last trigger to resolve has to be the one targeting Agrus, and the other five resolve before it in the order of your choosing. 

For our first example, we will have the trigger targeting Raiju resolve first, then the four triggers targeting Bears, and finally the one targeting Agrus.

Raiju gets a +1/+1 counter, no damage is dealt as Raiju only checks modified creatures other than himself. Total damage 0.
Bear #1 gets a +1/+1 counter, 1 damage is dealt as there is only one modified creature other than Raiju. Total damage 1.
Bear #2 gets a +1/+1 counter, 2 damage is dealt as there are now two modified creatures. Total damage 3.
Bear #3 gets a +1/+1 counter, 3 damage is dealt. Total damage 6.
Bear #4 gets a counter, 4 damage is dealt. Total damage of 10.
Agrus get a a counter, 5 damage is dealt. Total damage of 15.

Now let's have the four Bears' triggers resolve first, then Raiju, followed by Agrus.
Bear #1, 1 damage is dealt. Total damage 1.
Bear #2, 2 damage is dealt. Total damage 3.
Bear #3, 3 damage id dealt. Total damage 6.
Bear #4, 4 damage is dealt. Total damage is 10.
Raiju, 4 damage is dealt. Total damage is 14.
Agrus, 5 damage is dealt. Total damage is 19.

The later the trigger targeting Raiju resolves, the more the total damage.
 [/A]

[hr]
[cleft=Grafdigger's Cage]Alex, the answer is "What is used to 
keep the dead from escaping their graves?"[/cleft][Q] Does [c]Grafdigger's Cage[/c] stop [c]Abhorrent Oculus[/c] from manifest-dreading?[/Q]

[A] Well, in the important ways, yes. They still "manifest dread", but some parts they can't complete. Manifest Dread is "Look at the top two cards of your library. Manifest one of them, then put the cards you looked at that were not manifested this way into your graveyard." Since you can't put a creature from your library into play, both cards looked at are put into the graveyard.[/A]

[hr]
[Q] Last turn my opponent stole my [c]Bear Cub[/c] by casting [c]Sower of Temptation[/c]. This turn I used [c]Ovinize[/c] to make the Sower lose all its abilities. I did this because I think I'll get my Bear Cub back, since the Sower no longer has the "steal your creature" ability. My opponent disagrees. Who is right?[/Q]

[A] Your opponent is correct. Removing the enters ability (that creates an effect with a duration) after the ability has resolved, doesn't stop it from functioning.

If it helps, an example that seems more intuitive would be the following: My opponent used [c]Rootwater Matriarch[/c] to steel my [c]Hill Giant[/c] enchanted with [c]Spirit Link[/c]. Killing the Matriarch won't get the Hill Giant back, even though that removes where the steal ability came from. The duration is based on whether or not the Hill Giant is still enchanted.

So with the original question, removing Sower's abilities doesn't stop the steal effect. You have to remove the Sower to gain the creature back because that ends the duration of the effect.[/A]

[hr]
[Q] So continuing with the last question, what if instead of [c]Ovinize[/c]ing the [c]Sower of Temptation[/c] I phased it out with [c]Reality Ripple[/c]?[/Q]

[A] You would get your creature back. If a creature phases out, it's treated as though it's no longer on the battlefield. The duration of its effect has been met, so the effect ends and the stolen creature returns to its last controller, usually its owner.[/A]

[hr]
[Q] I block my opponent's attacking [c]Bear Cub[/c] with [c]Manor Gargoyle[/c]. After combat ends, the opponent then casts [c]Disfigure[/c] targeting my Gargoyle. They claim they just killed my Gargoyle. Is this true?[/Q]

[A] It is not true, your Gargoyle lives through this. Many new players make this same wrong assumption. Damage doesn't reduce toughness. You have a 2/2 Gargoyle with 2 damage marked on it. Because it's indestructible, the state based action that tries to destroy it for having equal or more damage than it has toughness fails to do anything. If they want to get your Gargoyle into the graveyard, they need to give it enough [c=Dismember]-X/-X to make its toughness 0 or less[/c], or force you to [c=Diabolic Edict]sacrifice it[/c].[/A]

[hr]
[Q] If [c]Ragavan, Nimble Pilferer[/c] is my commander, can I cast him for his dash cost from the command zone?[/Q]

[A] Yes, you can. Dash can be used from any zone, so long as you are allowed to cast that card from that same zone normally. [/A]

[hr]
[Q] Following up with the last question, do I have to pay commander tax if I dash out my commander?[/Q]

[A] Yes, you do. Suppose you cast Ragavan normally on turn one, then later he died and returned to the command zone. When you go to dash him he now costs [m]3[/m][m]R[/m] to dash. Then if he dies again and returns to the command zone, the next time you go to cast him he costs either [m]4[/m][m]R[/m] to cast normally or [m]5[/m][m]R[/m] to dash.[/A]

[hr]
[Q] I have eight cards in my hand. I go to my cleanup step and discard [c]Blazing Rootwalla[/c] to get down to maximum hand size. Can I cast the Rootwalla even though I'm in my cleanup step?[/Q]

[A] Yes, you can. During most cleanup steps, no players get priority. However, if an ability triggers during the cleanup step, or a state based action applies during a cleanup step, then the active player gains priority and may cast spells and activate abilities. 

This is when you can cast the Rootwalla, as discarding it to madness puts a trigger on the stack.

A normal priority pass will begin from here and until all players pass in succession with an empty stack, players may add new items to the stack when they have priority. Once all players pass with an empty stack, this cleanup step ends and a new one begins. If that new cleanup ends normally, then the game will move on to the next player's turn.[/A]

[hr]
[Q] While resolving [c]Burning Inquiry[/c] I discarded [c]Basking Rootwalla[/c] and two [c]Mountain[/c]s. My opponent controls a [c]Rest in Peace[/c]. My opponent claims I can't cast the Rootwalla for its Madness cost as it got exiled by Rest in Peace when I discarded it. Is this true?[/Q]

[A] Your opponent is wrong. You can cast your Rootwalla for its madness cost. The Rootwalla hasn't been exiled by Rest in Peace yet. This is because madness doesn't quite work the way most people think it does. If a card with madness gets discarded, it goes to exile instead of being put into its owner's graveyard. Then the next time a player would get priority, a trigger allowing its owner to cast it for its madness cost is put on the stack. When this trigger resolves, the owner may cast the card, but if they don't then the card is put into their graveyard. 

To get back to the original question, Rest in Peace can only exile a discarded Madness card after the owner has decided not to cast it for madness and choses to let it go to the graveyard.
[/A]

[hr]
[Q] My opponent pays [m]3[/m][m]U[/m] to gain level 2 on their [c]Stormchaser's Talent[/c]. Before it becomes level 2, I [c]Boomerang[/c] it to their hand. The opponent says they still get to return an instant or sorcery to their hand, I say they don't, who is right?[/Q]

[A] You are correct, they do not. Gaining a level is an ability that uses the stack and can be responded to. Stormchaser's level 2 ability is a trigger that triggers upon it becoming level 2 and happens only after it becomes level 2. If it's not on the battlefield when its level 2 ability resolves, there is no ability to trigger (nor a permanent to become level 2). [/A]

[hr]
[cright=Bear Cub|PO2]Alex, the answer is "What is another
name for Ursine Child?"[/cright][Q] I enchanted my opponent's [c]Bear Cub[/c] with my [c]Sigil of Sleep[/c]. If the Bear Cub goes unblocked attacking me, who chooses which creature I control gets returned to its owner's hand?[/Q]

[A] You get to choose as you control the Sigil of Sleep.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c] while I control [c]Anointed Procession[/c]. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] Both still get exiled. Saint Traft's triggered ability still made both tokens and tracks them to be exiled at the end of combat. Procession doesn't make tokens, it just changes how many tokens other cards and abilities make.[/A]

[hr]
[Q] I attack with [c]Geist of Saint Traft[/c], but before blocks, I cast [c]Eyes in the Skies[/c] choosing to populate the Angel token. When combat ends, will both Angel tokens get exiled or do I get to keep one?[/Q]

[A] You get to keep the one made by Eyes in the Skies. Unlike Anointed Procession from the last question, Eyes in the Skies makes the token and the token has no link to the ability that made the original Angel token. This includes that the second Angel token is not attacking, is not tapped, and it won't get exiled at the end of combat.[/A]

[hr]
[Q] I don't understand what [c]Emrakul, the World Anew[/c] has protection from. Can you clarify this?[/Q]

[A] Emrakul has protection from spells and separately protection from permanents cast "this turn". This turn is whichever turn it currently is, even if it's been many turns since Emrakul herself was cast. 

So for example, if I cast [c]Visara the Dreadful[/c] and gave her some [c]Lightning Greaves[/c] the same turn I cast her, she can't target and destroy Emrakul that turn. However, if I pass to my opponent's turn, Visara can now kill Emrakul as it's no longer the same turn Visara was cast. If I used [c]Goryo's Vengeance[/c] to reanimate Visara, then Visara could kill Emrakul that same turn as Visara wasn't cast at all.

Emrakul's protection ability acts similar to having protection from Auras. Auras when cast target what they will enchant, so Emrakul is safe from most Auras. However, on later turns, an Aura could be moved from its original target, to enchant Emrakul. For example, you could move [c]Pacifism[/c] from a 1/1 Goblin token to Emrakul using [c]Crown of the Ages[/c]. 
[/A]

[hr]
[Q] I cast [c]Season of Weaving[/c] choosing the two paws mode (to copy my [c]Arcanis the Omnipotent[/c]) and the three paws mode. Will I have to put one Arcanis into my graveyard due to the legend rule before bouncing all nonland and nontoken permanents?[/Q]

[A] You will not and you can not. The legend rule is a state based action. It's never checked in the middle of resolving a spell. You will put the original Arcanis into your hand before Season of Weaving fully resolves. The legend rule doesn't come into play during this situation.[/A]

[hr]
[Q] If I cast [c]Righteous Confluence[/c] choosing mode two once to exile my opponent's [c]Leyline of Sanctity[/c] and mode three twice to gain 10 total life, and the opponent responds by [c]Boomerang[/c]ing the Leyline back to their hand, do I still gain 10 life?[/Q]

[A] You do not. Righteous Confluence had only one target, Leyline of Sanctity, and that target has disappeared. If a spell loses all of its targets by the time it goes to resolve, it fizzles and none of its effects happen. [/A]

[hr]
[Q] I just cast [c]Cultivate[/c] putting a [c]Swamp[/c] onto the battlefield and a [c]Forest[/c] into my hand. I haven't played a land from my hand yet this turn. Can I still play the Forest as my land drop for the turn, or did the Swamp use up my land drop for the turn?[/Q]

[A] You may still play the Forest. Putting a land onto the battlefield with a spell or ability doesn't use up your one land drop per turn.[/A]

[hr]

Each Question in my last article had a hidden reference to a different pop-culture pig. With ten different questions there were ten different pigs. The spoiler below has the answer key to those references.
[spoiler]
Question 1) Charlotte's Web - Wilbur the Pig and Charlotte the Spider
Question 2) Jim Henson's Muppets - Miss Piggy, Kermit the Frog, and Fozzie Bear.
Question 3) The Three Little Pigs and the Big Bad Wolf
Question 4) Angry Birds Video Game (Birds vs Pigs)
Question 5) Looney Tunes - Bugs Bunny, Porky Pig, and Daffy Duck
Question 6) Babe the 1995 Movie - Babe the Pig, Sheep for Babe to herd, and Farmer Arthur Hoggett
Question 7) Legend of Zelda - Link an Elf Warrior, Zelda an Elf Noble, and Ganon (pig form of Ganondorf)
Question 8) The Lion King - Timon the Meerkat, Pumbaa the Warthog, and some Hyenas for them to fight.
Question 9) Toy Story - Woody the Cowboy, Buzz the Spaceman, Rex the T-Rex, and Hamm the Piggy Bank.
Question 10) Winnie the Pooh - Pooh the Bear, Tigger the Tiger, and Piglet the Pig.[/spoiler]

- Justin Hovdenes AKA Hovey
Level 2 Magic Judge
Rapid City, SD

{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'writeControl': {'requiredRevisionId': 'ALBJ4Lvf-aR1vWR3U4DLEgKLHARlbWPKDeFG9Gik11P3M39mcj5fxPa4nWODJwU3SaQ7NsirbZUjnVTEcvEyHQ'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'}
The Google doc at this URL has been modified with the desired content of length 15

----------

## Automatically send a short message to our Discord now that our Google doc has been set up for this weekend's translation job

First, I'll use my own private Discord server to try this out.
Then if everything works, I'll use our Discord server.

I'm following [this article](https://www.makerhacks.com/python-messages-discord/), after I created a webhook on my server by following [the official tutorial](https://support.discord.com/hc/en-us/articles/228383668-Intro-to-Webhooks).

In [101]:
# web library
import http.client

In [102]:
# your webhook URL
from google.colab import userdata
my_webhook_url = userdata.get('DISCORD_WEBHOOK')

def send_message_to_Discord(message, webhookurl=my_webhook_url):
    print(f"Starting to send the following message to my Discord server:\n{message}")

    # compile the form data (BOUNDARY can be anything)
    formdata = u"------:::BOUNDARY:::\r\nContent-Disposition: form-data; name=\"content\"\r\n\r\n" + message + u"\r\n------:::BOUNDARY:::--"

    # See https://stackoverflow.com/questions/74722329/how-to-send-accented-characters-with-diacritics-in-http-request-payload#74722361
    formdata = formdata.encode('utf-8')

    # get the connection and make the request
    connection = http.client.HTTPSConnection("discord.com")
    connection.request("POST", webhookurl, formdata, {
        'content-type': "multipart/form-data; boundary=----:::BOUNDARY:::",
        'cache-control': "no-cache",
    })

    # get the response
    response = connection.getresponse()
    result = response.read()

    # return back to the calling function with the result
    result_decoded_in_utf8 = result.decode("utf-8")
    print("Success!")
    if result_decoded_in_utf8:
        print(result_decoded_in_utf8)
    return result_decoded_in_utf8

In [103]:
send_message_to_Discord(u"*Hello, world!*\n> Sent from a Google colab notebook running Python! Code by Naereen")

Starting to send the following message to my Discord server:
*Hello, world!*
> Sent from a Google colab notebook running Python! Code by Naereen
Success!


''

--------------------------

## Do it all in one function: download article, translate it (if running on GPU), upload to GDoc, notify the team, make coffee!

In [104]:
def do_all_then_notifyme(article_id=None, translate=False):
    if article_id is None:
        article_id = get_latest_article_id()
    print(f"- The latest article ID is: {article_id}... Downloading it...")
    if translate:
        print("Then translating it...")
    print("Then uploading it to our Google doc...")

    latest_article_title, latest_article_date = get_article_translate_it_then_upload_to_google_doc(article_id, translate=translate)

    latest_article_link = f"https://www.cranial-insertion.com/article/{article_id}"

    message_for_Discord = f"""||@everyone|| [Notre Google doc partagé]({our_google_doc_url}) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire]({latest_article_link}) de [Cranial Insertion](https://www.cranial-insertion.com).

Cet article s'intitule "{latest_article_title}", a été publié le {latest_article_date}, et son ID est {article_id}.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab ! Code by @naereen
"""
    print_paragraph(message_for_Discord)

    return send_message_to_Discord(message_for_Discord)

### Let's try this « do it all » function!

In [106]:
%%time

do_all_then_notifyme(article_id=4371, translate=running_on_GPU)

- The latest article ID is: 4371... Downloading it...
Then uploading it to our Google doc...
- The latest article ID is: 4371... Downloading it, then translating it...
Reading the article at URL https://www.cranial-insertion.com/staff/articles/4371/edit ...
Using French language and my editor cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !
The 1-th card found is withing a tag « [c]Hardened Scales[/c] », named « Hardened Scales »...
  => it seems to be translated (to French) into « Écailles renforcées »
The 2-th card found is withing a tag « [c]Bounty of the Hunt[/c] », named « Bounty of the Hunt »...
  => it seems to be translated (to French) into « Prime de chasse »
The 3-th card found is withing a tag « [c]Spelunking[/c] », named « Spelunking »...
  => it seems to be translated (to French) into « Spéléologie »
The 4-th card found is withing a tag « [c]Titania, Gaea Incarnate[/c] », named « Titania, Gaea Incarnate »...
  

0it [00:00, ?it/s]



0-th English paragraph (of length 577 letters and 94 words)


> [cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.



1-th English paragraph (of length 257 letters and 42 words)


...

> # TITLE: « A Very Green Day »

Une suggestion automatique de traduction de ce titre est :

## TITRE : « Une journée très verte »

- Cet article vient de cette page web : https://www.cranial-insertion.com/article/4371
- Il a été ou sera publié le : 17/03/2025

--------------------------------------------------------
    

[cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.

As always, if you have questions for us, you can email them to us at [email]moko@cranialinsertion.com[/email], or tweet short questions to our X account @CranialTweet. One of our authors will reply to you, and your question might appear in a future article.

[hr]
[q]Let's say I control [c=Hardened Scales]Écailles renforcées[/c] and I cast [c=Bounty of the Hunt]Prime de chasse[/c], assigning one counter to each of three target creatures. Because of Hardened Scales, this will result in two counters being put onto each target. Do I have to remove all six counters or just one counter per creature at the beginning of the cleanup step?[/q]

[a]I'm afraid you'll have to remove all six counters. Bounty of the Hunt tracks how many counters each creature received "this way", which means how many counters it received due to following the instructions of Bounty of the Hunt as modified by any applicable replacement effects. Since each target creature received two counters this way, you'll have to remove two counters from each of them.[/a]

[hr]
[q]If I control [c=Spelunking]Spéléologie[/c] and [c=Titania, Gaea Incarnate]Titania, Voix de Gaia[/c] enters, do the lands return from the graveyard tapped or untapped?[/q]

[a]The lands will enter untapped unless the land itself has a replacement effect that says it enters tapped, in which case you get a choice between tapped and untapped. Titania's effect instructing you to return the lands to the battlefield tapped is not a replacement effect, it's just the effect of what the ability does. If the land doesn't have a replacement effect itself, then Spelunking's effect is the only replacement effect involved, and it replaces Titania's "tapped" with "untapped". If the land has a replacement effect along the lines of "this land enters tapped", then you choose the order in which to apply the land's replacement effect and Spelunking's effect, which results in a choice between tapped and untapped.[/a]

[hr]
[q]If I control [c=Ashaya, Soul of the Wild]Ashaya, âme de la nature[/c], can I put creatures onto the battlefield with [c=Cultivator Colossus]Colosse cultivateur[/c]'s ability because they're also lands?[/q]

[a]No, that doesn't work. Ashaya's ability only affects nontoken creatures that are on the battlefield. It does not affect the creature cards that are in your hand.[/a]

[hr]
[q]I control a [c=Plaxcaster Frogling]Grenouilleau lanceplax[/c] that still has a +1/+1 counter left on it and my opponent tries to [c=Murder]Homicide[/c] it. If I activate the Frogling's ability to give it shroud, does that make Murder fizzle?[/q]

[a]Absolutely. Plaxcaster Frogling can target itself with its ability, and the ability resolves first, giving the Frogling shroud. When Murder tries to resolve, it finds that its only target has become an illegal target, so it doesn't resolve and just goes to the graveyard.[/a]

[hr]
[q]If I use [c]Biophagus[/c]'s mana to cast [c=Nylea, God of the Hunt]Nyléa, déesse de la Chasse[/c] but I don't have enough devotion to green for Nylea to be a creature, does it still get the +1/+1 counter?[/q]

[a]Yup! Since Nylea is by default a creature and the ability that turns off its creature-ness only functions on the battlefield, you are in fact using Biophagus's mana to cast a creature spell, so Nylea gets the replacement effect "enters with an additional +1/+1 counter" stapled onto it. There is nothing stopping a noncreature permanent from entering with a +1/+1 counter, and there is no reason for a +1/+1 counter to fall off from a noncreature permanent. The counter will be placed on Nylea and stays there, it just won't have any effect until Nylea is a creature.[/a]

[hr]
[q]If [c=Displaced Dinosaurs]Dinosaures anachroniques[/c] makes a planeswalker creature and that planeswalker creature gets attacked, can it block the attacker?[/q]

[a]Sure, that's perfectly legal and probably beneficial even though it does not change the type or amount of harm that's coming to the planeswalker. Whether you block the attacker or not, it will deal its combat damage to the planeswalker creature, and in either case this will cause the planeswalker creature to lose loyalty counters (because it's a planeswalker) and to have damage marked on it (because it's a creature). However, as a blocking creature the planeswalker creature gets to deal combat damage back to the attacker, and as a 7/7 Dinosaur it deals a pretty significant amount of damage.[/a]

[hr]
[cleft=Wild Dogs]Hear the dogs howling out of key
To a hymn called "Faith and Misery"[/cleft][q]What does [c=Wild Dogs]Chiens sauvages[/c]'s triggered ability do in Two-Headed Giant?[/q]

[a]It does very little. One might think that a player on the team with the greater life total would gain control of the Dogs, but that's not how this works. The ability checks before it even goes on the stack whether an individual [i]player[/i] has more life than each other individual player. When an effect in Two-Headed Giant asks for a player's life total, the answer is that player's team's shared life total, so it will always get the same number for the two players on the same team. This means that the condition the ability looks for can never be true in Two-Headed Giant, so the ability never goes on the stack.[/a]

[hr]
[q]What are the card types that [c=Winter, Cynical Opportunist]Frimas, opportuniste cynique[/c] looks for?[/q]

[a]The term "card type" refers to the "main" card type of a card, which includes things like artifact, creature, land, and so on. On the type line there may be a supertype such as legendary or basic in front of the card type; those are not card types, so Winter isn't interested in those. After the dash you'll find subtypes such as Elf, Forest, or Equipment. Those aren't card types either, so Winter isn't interested in those, either.[/a]

[hr]
[q]Can [c=Winter, Cynical Opportunist]Frimas, opportuniste cynique[/c]'s ability put a battle's back face onto the battlefield? For example, can I put [c=Invasion of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c] onto the battelfield as [c=Zilortha, Apex of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c]?[/q]

[a]No. When an effect puts a double-faced card onto the battlefield without specifying a face, it is put onto the battlefield with its front face up.[/a]

[hr]
[q]Does [c=Garruk's Packleader]Chef de meute de Garruk[/c] see itself entering?[/q]

[a]A little bit yes, but mostly no. Every creature sees itself entering the battlefield, since enters abilities are checked after the event, when the creature is on the battlefield. However, Garruk's Packleader's ability only triggers whenever [i]another[/i] creature with power 3 or greater enters, so it doesn't trigger its own enters ability.[/a]

[hr]
[q]I'm in a game of Commander, my [c]Ms. Bumbleflower[/c]'s ability has triggered, but all my opponents have hexproof. Can I still give a creature a +1/+1 counter and flying?[/q]

[a]No. In order to put Ms. Bumbleflower's ability on the stack, you have to choose legal targets for each required target. The ability needs a target opponent and a target creature, and since you can't legally target an opponent, the ability is immediately removed from the stack.[/a]

[hr]
[q]I am casting [c=Reborn Hope]Espoir renaissant[/c] off of a [c=Counterpoint]Contrepoint[/c] that targeted a spell with mana value 2 or greater. Can Reborn Hope bring back this Counterpoint?[/q]

[a]No. You have to choose the target for Reborn Hope at the moment you cast it, and you're casting Reborn Hope while Counterpoint is resolving. At that moment, Counterpoint is on the stack, not in the graveyard, so you can't target it with Reborn Hope.[/a]

[hr]
[cright=Deathrite Shaman]I pledge allegiance to the underworld[/cright][q]Does [c]Niv-Mizzet, Guildpact[/c] count hybrid one-drops such as [c=Judge's Familiar]Familier du juge[/c] or [c=Deathrite Shaman]Shamane ritemort[/c] towards its X?[/q]

[a]Absolutely. The hybrid mana symbol has both of its colors, which means that the permanent is both of those colors. Deathrite Shaman is both black and green, and Judge's Familiar is both white and blue, so they each count towards Niv-Mizzet's X.[/a]

[hr]
[q]If I copy [c=Wildwood Scourge]Fléau de bois sauvage[/c] with X=5, does the copy also enter with five counters?[/q]

[a]That depends on how you copy it. If you make a token copy of a Wildwood Scourge that's already on the battlefield, for example with [c=Spitting Image]Portrait craché[/c], then no. The copy's X is 0, so it enters as a sad 0/0 without any +1/+1 counters on it. However, if Wildwood Scourge is a spell on the stack and you're copying it with [c]Magus Lucea Kane[/c]'s ability, then the copied spell has the same X as the original, and the resulting token will enter with the same number of counters.[/a]

[hr]
[q]If [c=Maze of Ith]Labyrinthe d'Ith[/c] gets an everything counter from [c=Omo, Queen of Vesuva]Omo, reine de Vésuva[/c], can I tap it for green mana?[/q]

[a]Sure, as long as Omo sticks around. Omo's ability gives Maze of Ith every land type from Cave to Urza's and everything in between, including Forest. As a consequence of becoming a Forest, Maze of Ith gains the intrinsic ability to produce green mana. [/a]

[hr]
[q]Does [c=Choking Vines]Vignes étrangleuses[/c] do anything against an attacker with trample such as [c=Giant Warthog]Phacochère géant[/c]?[/q]

[a]Not really. While the Warthog becomes blocked due to Choking Vines, there is no creature there that's blocking it when combat damage is assigned. As we've seen last week, this means that all its damage gets assigned to the thing it's attacking, which is exactly what would have happened if you hadn't cast Choking Vines.[/a]

[hr]
[q]Is putting [c=Bigger on the Inside]Plus grand à l'intérieur[/c] on [c]Pili-Pala[/c] as ridiculous as I think it is?[/q]

[a]If you think that this gives you an arbitrary amount of mana and cascade triggers, you're exactly right! Because of Bigger on the Inside, you can tap Pili-Pala to create two mana and to give your next spell cascade. You can use those two mana to untap Pili-Pala for its own ability to make one mana, and now you're back to the same game state where you started except for an extra mana and an extra cascade ability on your next spell. Lather, rinse, repeat as needed.[/a]

[hr]
[q]Does [c=Insidious Roots]Racines insidieuses[/c]'s ability get triggered by [c=Ojer Kaslem, Deepest Growth]Ojer Kaslem, Croissance des profondeurs // Temple de la culture[/c] returning from the graveyard transformed? Or by [c=Mosswood Dreadknight // Dread Whispers]Mosswood Dreadknight[/c] being cast from my graveyard as an adventure? Or by [c=Enduring Vitality]Vitalité perpétuelle[/c] returning as an enchantment?  [/q]

[a]The answer is yes for all three scenarios. In each one, you have a creature card leaving your graveyard, and that's all that Insidious Roots cares about. It doesn't matter what zone the creature card goes to or whether it's still a creature by the time it gets there.[/a]

[hr]

And that's all the time we have for today. Thanks for reading, I hope you have a safe and happy Saint Patrick's Day, and please come back next week for more [b]Magic[/b] rules Q&A.

- Carsten Haese


{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'writeControl': {'requiredRevisionId': 'ALBJ4Ls40iDpryBuSEF5PqZehoksne6ZTWH0FZgAhW4jm7PZOCuj46fxVS50_ka79n-VSf1aNAb4xuyowdc5Jw'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'}
The Google doc at this URL has been modified with the desired content of length 12148:
https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing


> ||@everyone|| [Notre Google doc partagé](https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire](https://www.cranial-insertion.com/article/4371) de [Cranial Insertion](https://www.cranial-insertion.com).

Cet article s'intitule "A Very Green Day", a été publié le 03/17/2025, et son ID est 4371.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab ! Code by @naereen


Starting to send the following message to my Discord server:
||@everyone|| [Notre Google doc partagé](https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire](https://www.cranial-insertion.com/article/4371) de [Cranial Insertion](https://www.cranial-insertion.com).

Cet article s'intitule "A Very Green Day", a été publié le 03/17/2025, et son ID est 4371.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab ! Code by @naereen

Success!
CPU times: user 1.29 s, sys: 60.1 ms, total: 1.35 s
Wall time: 3.16 s


''

### Let's try to translate the latest article, on the CPU (free):

In [107]:
%%time
do_all_then_notifyme(article_id=4371, translate=True)

- The latest article ID is: 4371... Downloading it...
Then translating it...
Then uploading it to our Google doc...
- The latest article ID is: 4371... Downloading it, then translating it...
Reading the article at URL https://www.cranial-insertion.com/staff/articles/4371/edit ...
Using French language and my editor cookie...
The web page has been read and it's title is « <title>Cranial Insertion | Manage Articles</title> » !
The 1-th card found is withing a tag « [c]Hardened Scales[/c] », named « Hardened Scales »...
  => it seems to be translated (to French) into « Écailles renforcées »
The 2-th card found is withing a tag « [c]Bounty of the Hunt[/c] », named « Bounty of the Hunt »...
  => it seems to be translated (to French) into « Prime de chasse »
The 3-th card found is withing a tag « [c]Spelunking[/c] », named « Spelunking »...
  => it seems to be translated (to French) into « Spéléologie »
The 4-th card found is withing a tag « [c]Titania, Gaea Incarnate[/c] », named « Titania,

0it [00:00, ?it/s]



0-th English paragraph (of length 577 letters and 94 words)


> [cright=Fork in the road]I walk a lonely road
The only one that I have ever known
[/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.

Translated 0-th paragraph in French (of length 613 letters and 97 words)


...

> # TITLE: « A Very Green Day »

Une suggestion automatique de traduction de ce titre est :

## TITRE : « Une journée très verte »

- Cet article vient de cette page web : https://www.cranial-insertion.com/article/4371
- Il a été ou sera publié le : 17/03/2025

--------------------------------------------------------
    

    [cright=Fork in the road]I walk a lonely road
    The only one that I have ever known
    [/cright]Greetings and welcome back to another issue of Cranial Insertion! Today is Saint Patrick's Day in Ireland and other countries around the world, so people are celebrating Irish culture in a number of ways, including wearing green clothes. Neither Moko nor I are Irish, but Moko is getting into the Saint Patrick's Day spirit and has decorated the Cranial Insertion offices with green tinsel and shamrock stickers, and I have decided to mark the occasion by turning today's article green.

[crightFork in the road]Je marche sur une route solitaire La seule que j'aie jamais connue [/cright]Salutations et bienvenue à un autre numéro de Cranial Insertion! Aujourd'hui, c'est la Saint-Patrick en Irlande et dans d'autres pays du monde, donc les gens célèbrent la culture irlandaise de plusieurs façons, y compris en portant des vêtements verts.Ni Moko ni moi ne sommes Irlandais, mais Moko se met dans l'esprit de la Saint-Patrick et a décoré les bureaux de Cranial Insertion avec des stickers en guirlande verte et trèfle, et j'ai décidé de marquer l'occasion en tournant l'article d'aujourd'hui au vert.

    As always, if you have questions for us, you can email them to us at [email]moko@cranialinsertion.com[/email], or tweet short questions to our X account @CranialTweet. One of our authors will reply to you, and your question might appear in a future article.

Comme toujours, si vous avez des questions pour nous, vous pouvez nous les envoyer par courriel à [email]mokocranialinsertion.com[/email], ou tweeter de courtes questions à notre compte X CranialTweet. Un de nos auteurs vous répondra, et votre question pourrait apparaître dans un prochain article.

    [hr]
    [q]Let's say I control [c=Hardened Scales]Écailles renforcées[/c] and I cast [c=Bounty of the Hunt]Prime de chasse[/c], assigning one counter to each of three target creatures. Because of Hardened Scales, this will result in two counters being put onto each target. Do I have to remove all six counters or just one counter per creature at the beginning of the cleanup step?[/q]

[hr] [q]Disons que je contrôle [cHardened Scales]Écailles renforcées[/c] et que je lance [cBounty of the Hunt]Prime de chasse[/c] en attribuant un marqueur à chacune des trois créatures cibles..En raison des échelles durcies, deux marqueurs seront placés sur chaque cible..Dois-je retirer les six marqueurs ou juste un marqueur par créature au début de l'étape de nettoyage ?[/q]

    [a]I'm afraid you'll have to remove all six counters. Bounty of the Hunt tracks how many counters each creature received "this way", which means how many counters it received due to following the instructions of Bounty of the Hunt as modified by any applicable replacement effects. Since each target creature received two counters this way, you'll have to remove two counters from each of them.[/a]

J'ai peur que vous deviez retirer les six compteurs.Bounty of the Hunt suit le nombre de marqueurs que chaque créature a reçu "de cette façon", ce qui signifie le nombre de marqueurs qu'elle a reçu en raison de suivre les instructions de Bounty of the Hunt telles que modifiées par tout effet de remplacement applicable..Puisque chaque créature cible a reçu deux marqueurs de cette façon, vous devrez retirer deux marqueurs de chacun d'eux..[/a]

    [hr]
    [q]If I control [c=Spelunking]Spéléologie[/c] and [c=Titania, Gaea Incarnate]Titania, Voix de Gaia[/c] enters, do the lands return from the graveyard tapped or untapped?[/q]

[hr] [q]Si je contrôle [cSpelunking]Spéléologie[/c] et [cTitania, Gaea Incarnate]Titania, Voix de Gaia[/c] entre, est-ce que les terres reviennent du cimetière tapé ou inexploité?[/q]

    [a]The lands will enter untapped unless the land itself has a replacement effect that says it enters tapped, in which case you get a choice between tapped and untapped. Titania's effect instructing you to return the lands to the battlefield tapped is not a replacement effect, it's just the effect of what the ability does. If the land doesn't have a replacement effect itself, then Spelunking's effect is the only replacement effect involved, and it replaces Titania's "tapped" with "untapped". If the land has a replacement effect along the lines of "this land enters tapped", then you choose the order in which to apply the land's replacement effect and Spelunking's effect, which results in a choice between tapped and untapped.[/a]

Les terres entreront inexploitées à moins que la terre elle-même n'ait un effet de remplacement qui dit qu'elle entre inexploitée, auquel cas vous avez le choix entre inexploitée et inexploitée..L'effet de Titania vous demandant de retourner les terres sur le champ de bataille n'est pas un effet de remplacement, c'est juste l'effet de ce que fait la capacité..Si la terre n'a pas d'effet de remplacement elle-même, alors l'effet de Spelunking est le seul effet de remplacement impliqué, et il remplace Titania par "inexploité"..Si la terre a un effet de remplacement le long des lignes de "cette terre entre exploitée", alors vous choisissez l'ordre dans lequel appliquer l'effet de remplacement de la terre et l'effet de Spelunking, qui se traduit par un choix entre exploité et inexploité.[/a]

    [hr]
    [q]If I control [c=Ashaya, Soul of the Wild]Ashaya, âme de la nature[/c], can I put creatures onto the battlefield with [c=Cultivator Colossus]Colosse cultivateur[/c]'s ability because they're also lands?[/q]

[hr] [q]Si je contrôle [cAshaya, Soul of the Wild]Ashaya, âme de la nature[/c], puis-je mettre des créatures sur le champ de bataille avec la capacité de [cCultivator Colossus]Colosse cultivateur[/c] parce qu'elles sont aussi des terres ?[/q]

    [a]No, that doesn't work. Ashaya's ability only affects nontoken creatures that are on the battlefield. It does not affect the creature cards that are in your hand.[/a]

[a]Non, cela ne fonctionne pas. La capacité d'Ashaya n'affecte que les créatures non-jetonnes qui sont sur le champ de bataille. Cela n'affecte pas les cartes de créature qui sont dans votre main.[/a]

    [hr]
    [q]I control a [c=Plaxcaster Frogling]Grenouilleau lanceplax[/c] that still has a +1/+1 counter left on it and my opponent tries to [c=Murder]Homicide[/c] it. If I activate the Frogling's ability to give it shroud, does that make Murder fizzle?[/q]

[hr] [q]Je contrôle un [cPlaxcaster Frogling]Grenouilleau lanceplax[/c] qui a encore un marqueur +1/+1 à gauche et mon adversaire essaie de [cMurder]Homicide[/c]. Si j'active la capacité du Frogling à lui donner un linceul, est-ce que cela rend Murder fizzle?[/q]

    [a]Absolutely. Plaxcaster Frogling can target itself with its ability, and the ability resolves first, giving the Frogling shroud. When Murder tries to resolve, it finds that its only target has become an illegal target, so it doesn't resolve and just goes to the graveyard.[/a]

[a] Absolument. Plaxcaster Frogling peut se cibler avec sa capacité, et la capacité se résout en premier, donnant le linceul Frogling. Lorsque Murder essaie de résoudre, il trouve que sa seule cible est devenue une cible illégale, donc il ne résout pas et va juste au cimetière.

    [hr]
    [q]If I use [c]Biophagus[/c]'s mana to cast [c=Nylea, God of the Hunt]Nyléa, déesse de la Chasse[/c] but I don't have enough devotion to green for Nylea to be a creature, does it still get the +1/+1 counter?[/q]

[hr] [q]Si j'utilise le mana de [c]Biophagus[/c] pour lancer [cNylea, God of the Hunt]Nyléa, déesse de la Chasse[/c] mais que je n'ai pas assez de dévotion au vert pour que Nylea soit une créature, est-ce qu'il obtient quand même le marqueur +1/+1 ?[/q]

    [a]Yup! Since Nylea is by default a creature and the ability that turns off its creature-ness only functions on the battlefield, you are in fact using Biophagus's mana to cast a creature spell, so Nylea gets the replacement effect "enters with an additional +1/+1 counter" stapled onto it. There is nothing stopping a noncreature permanent from entering with a +1/+1 counter, and there is no reason for a +1/+1 counter to fall off from a noncreature permanent. The counter will be placed on Nylea and stays there, it just won't have any effect until Nylea is a creature.[/a]

Étant donné que Nylea est par défaut une créature et que la capacité qui désactive sa créature ne fonctionne que sur le champ de bataille, vous utilisez en fait le mana de Biophagus pour lancer un sort de créature, de sorte que Nylea obtient l'effet de remplacement "entre avec un compteur supplémentaire +1/+1" agrafé dessus.Il n'y a rien qui empêche un permanent non-créature d'entrer avec un compteur +1/+1, et il n'y a aucune raison pour qu'un compteur +1/+1 tombe d'un permanent non-créature..Le compteur sera placé sur Nylea et restera là, il n'aura aucun effet jusqu'à ce que Nylea soit une créature..[/a]

    [hr]
    [q]If [c=Displaced Dinosaurs]Dinosaures anachroniques[/c] makes a planeswalker creature and that planeswalker creature gets attacked, can it block the attacker?[/q]

[hr] [q]Si [cDisplaced Dinosaurs]Dinosaures anachroniques[/c] fait une créature planeswalker et que cette créature planeswalker est attaquée, peut-elle bloquer l'attaquant ?[/q]

    [a]Sure, that's perfectly legal and probably beneficial even though it does not change the type or amount of harm that's coming to the planeswalker. Whether you block the attacker or not, it will deal its combat damage to the planeswalker creature, and in either case this will cause the planeswalker creature to lose loyalty counters (because it's a planeswalker) and to have damage marked on it (because it's a creature). However, as a blocking creature the planeswalker creature gets to deal combat damage back to the attacker, and as a 7/7 Dinosaur it deals a pretty significant amount of damage.[/a]

Bien sûr, c'est parfaitement légal et probablement bénéfique même si cela ne change pas le type ou la quantité de mal qui vient au planeswalker..Que vous bloquiez l'attaquant ou non, il infligera ses dégâts de combat à la créature planeswalker, et dans les deux cas, la créature planeswalker perdra ses marqueurs de loyauté (parce que c'est un planeswalker) et les dégâts seront marqués dessus (parce que c'est une créature)..Cependant, en tant que créature bloquante, la créature planeswalker inflige des dégâts de combat à l'attaquant, et en tant que dinosaure 7/7, elle inflige une quantité assez importante de dégâts..[/a]

    [hr]
    [cleft=Wild Dogs]Hear the dogs howling out of key
    To a hymn called "Faith and Misery"[/cleft][q]What does [c=Wild Dogs]Chiens sauvages[/c]'s triggered ability do in Two-Headed Giant?[/q]

[hr] [cleftWild Dogs]Entendez les chiens hurler hors de la clé À un hymne appelé "Faith and Misery"[/cleft][q]Qu'est-ce que la capacité déclenchée de [cWild Dogs]Chiens sauvages[/c] fait dans Two-Headed Giant?[/q]

    [a]It does very little. One might think that a player on the team with the greater life total would gain control of the Dogs, but that's not how this works. The ability checks before it even goes on the stack whether an individual [i]player[/i] has more life than each other individual player. When an effect in Two-Headed Giant asks for a player's life total, the answer is that player's team's shared life total, so it will always get the same number for the two players on the same team. This means that the condition the ability looks for can never be true in Two-Headed Giant, so the ability never goes on the stack.[/a]

[A] Il fait très peu.On pourrait penser qu'un joueur de l'équipe avec le plus grand total de vie prendrait le contrôle des Chiens, mais ce n'est pas ainsi que cela fonctionne..La capacité vérifie avant même d'aller sur la pile si un joueur individuel a plus de vie que chaque autre joueur individuel..Lorsqu'un effet dans Géant à deux têtes demande le total de vie d'un joueur, la réponse est le total de vie partagée de l'équipe de ce joueur, de sorte qu'il obtiendra toujours le même nombre pour les deux joueurs de la même équipe..Cela signifie que la condition que la capacité recherche ne peut jamais être vraie dans le Géant à deux têtes, de sorte que la capacité ne va jamais sur la pile..[/a]

    [hr]
    [q]What are the card types that [c=Winter, Cynical Opportunist]Frimas, opportuniste cynique[/c] looks for?[/q]

[hr] [q]Quels sont les types de cartes que [cWinter, Opportuniste cynique]Frimas, opportuniste cynique[/c] recherche ?[/q]

    [a]The term "card type" refers to the "main" card type of a card, which includes things like artifact, creature, land, and so on. On the type line there may be a supertype such as legendary or basic in front of the card type; those are not card types, so Winter isn't interested in those. After the dash you'll find subtypes such as Elf, Forest, or Equipment. Those aren't card types either, so Winter isn't interested in those, either.[/a]

[a] Le terme "type de carte" fait référence au type de carte "principal" d'une carte, qui comprend des objets tels qu'un artefact, une créature, une terre, etc..Sur la ligne de type, il peut y avoir un supertype tel que légendaire ou basique devant le type de carte; ce ne sont pas des types de carte, donc Winter n'est pas intéressé par ceux.Après le tableau de bord, vous trouverez des sous-types tels que Elf, Forest ou Equipment..Ce ne sont pas des types de cartes non plus, donc l'hiver n'est pas intéressé par ceux-là non plus..[/a]

    [hr]
    [q]Can [c=Winter, Cynical Opportunist]Frimas, opportuniste cynique[/c]'s ability put a battle's back face onto the battlefield? For example, can I put [c=Invasion of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c] onto the battelfield as [c=Zilortha, Apex of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c]?[/q]

[hr] [q]La capacité de [cWinter, Cynical Opportunist]Frimas, opportuniste cynique[/c] peut-elle mettre la face arrière d'une bataille sur le champ de bataille ? Par exemple, puis-je mettre [cZilexion of Ikoria]Invasion of Ikoria // Zilortha, Apex of Ikoria[/c] sur le champ de bataille ?

    [a]No. When an effect puts a double-faced card onto the battlefield without specifying a face, it is put onto the battlefield with its front face up.[/a]

[a] Non. Quand un effet met une carte à double face sur le champ de bataille sans spécifier de visage, il est mis sur le champ de bataille avec son front face visible.[/a]

    [hr]
    [q]Does [c=Garruk's Packleader]Chef de meute de Garruk[/c] see itself entering?[/q]

[hr] [q]Est-ce que [cGarruk's Packleader]Chef de meute de Garruk[/c] se voit entrer?[/q]

    [a]A little bit yes, but mostly no. Every creature sees itself entering the battlefield, since enters abilities are checked after the event, when the creature is on the battlefield. However, Garruk's Packleader's ability only triggers whenever [i]another[/i] creature with power 3 or greater enters, so it doesn't trigger its own enters ability.[/a]

[a]Un peu oui, mais la plupart du temps non. Chaque créature se voit entrer sur le champ de bataille, puisque les capacités d'entrée sont vérifiées après l'événement, lorsque la créature est sur le champ de bataille. Cependant, la capacité du chef de meute de Garruk ne se déclenche que chaque fois qu'une autre créature avec un pouvoir de 3 ou plus entre, de sorte qu'elle ne déclenche pas sa propre capacité d'entrée.

    [hr]
    [q]I'm in a game of Commander, my [c]Ms. Bumbleflower[/c]'s ability has triggered, but all my opponents have hexproof. Can I still give a creature a +1/+1 counter and flying?[/q]

[hr] [q]Je suis dans une partie de Commander, ma capacité de [c]Mme Bumbleflower[/c] s'est déclenchée, mais tous mes adversaires sont à l'épreuve de l'hexagone. Puis-je quand même donner à une créature un marqueur +1/+1 et voler ?[/q]

    [a]No. In order to put Ms. Bumbleflower's ability on the stack, you have to choose legal targets for each required target. The ability needs a target opponent and a target creature, and since you can't legally target an opponent, the ability is immediately removed from the stack.[/a]

[a]Non. Afin de mettre la capacité de Mme Bumbleflower sur la pile, vous devez choisir des cibles légales pour chaque cible requise. La capacité a besoin d'un adversaire cible et d'une créature cible, et puisque vous ne pouvez pas légalement cibler un adversaire, la capacité est immédiatement retirée de la pile.[/a]

    [hr]
    [q]I am casting [c=Reborn Hope]Espoir renaissant[/c] off of a [c=Counterpoint]Contrepoint[/c] that targeted a spell with mana value 2 or greater. Can Reborn Hope bring back this Counterpoint?[/q]

[hr] [q]Je lance [cReborn Hope]Espoir renaissant[/c] hors d'un [cCounterpoint]Contrepoint[/c] qui a ciblé un sort avec une valeur de mana de 2 ou plus.

    [a]No. You have to choose the target for Reborn Hope at the moment you cast it, and you're casting Reborn Hope while Counterpoint is resolving. At that moment, Counterpoint is on the stack, not in the graveyard, so you can't target it with Reborn Hope.[/a]

[a]Non. Vous devez choisir la cible pour Reborn Hope au moment où vous la lancez, et vous lancez Reborn Hope pendant que Counterpoint est résolu. À ce moment-là, Counterpoint est sur la pile, pas dans le cimetière, donc vous ne pouvez pas le cibler avec Reborn Hope.[/a]

    [hr]
    [cright=Deathrite Shaman]I pledge allegiance to the underworld[/cright][q]Does [c]Niv-Mizzet, Guildpact[/c] count hybrid one-drops such as [c=Judge's Familiar]Familier du juge[/c] or [c=Deathrite Shaman]Shamane ritemort[/c] towards its X?[/q]

[hr] [crightDeathrite Shaman]Je jure allégeance à la pègre[/cright][q]Est-ce que [c]Niv-Mizzet, Guildpact[/c] compte une goutte hybride comme [cJudge's Familiar]Familier du juge[/c] ou [cDeathrite Shaman]Shamane ritemort[/c] vers son X?

    [a]Absolutely. The hybrid mana symbol has both of its colors, which means that the permanent is both of those colors. Deathrite Shaman is both black and green, and Judge's Familiar is both white and blue, so they each count towards Niv-Mizzet's X.[/a]

[a] Absolument. Le symbole de mana hybride a ses deux couleurs, ce qui signifie que le permanent est ces deux couleurs. Deathrite Shaman est à la fois noir et vert, et Judge's Familiar est à la fois blanc et bleu, donc ils comptent chacun pour le X de Niv-Mizzet.

    [hr]
    [q]If I copy [c=Wildwood Scourge]Fléau de bois sauvage[/c] with X=5, does the copy also enter with five counters?[/q]

[hr] [q]Si je copie [cWildwood Scourge]Fléau de bois sauvage[/c] avec X5, la copie entre-t-elle aussi avec cinq compteurs ?[/q]

    [a]That depends on how you copy it. If you make a token copy of a Wildwood Scourge that's already on the battlefield, for example with [c=Spitting Image]Portrait craché[/c], then no. The copy's X is 0, so it enters as a sad 0/0 without any +1/+1 counters on it. However, if Wildwood Scourge is a spell on the stack and you're copying it with [c]Magus Lucea Kane[/c]'s ability, then the copied spell has the same X as the original, and the resulting token will enter with the same number of counters.[/a]

[a]Cela dépend de la façon dont vous le copiez.Si vous faites une copie symbolique d'un Fléau de Wildwood qui est déjà sur le champ de bataille, par exemple avec [cSpitting Image]Portrait craché[/c], alors non.Le X de la copie est 0, donc il entre comme un triste 0/0 sans aucun compteur +1/+1 dessus.Cependant, si Wildwood Scourge est un sort sur la pile et que vous le copiez avec la capacité de [c]Magus Lucea Kane[/c], alors le sort copié a le même X que l'original, et le jeton résultant entrera avec le même nombre de marqueurs..[/a]

    [hr]
    [q]If [c=Maze of Ith]Labyrinthe d'Ith[/c] gets an everything counter from [c=Omo, Queen of Vesuva]Omo, reine de Vésuva[/c], can I tap it for green mana?[/q]

[hr] [q]Si [cMaze of Ith]Labyrinthe d'Ith[/c] obtient un compteur de tout de [cOmo, Queen of Vesuva]Omo, reine de Vésuva[/c], puis-je le taper pour le mana vert?[/q]

    [a]Sure, as long as Omo sticks around. Omo's ability gives Maze of Ith every land type from Cave to Urza's and everything in between, including Forest. As a consequence of becoming a Forest, Maze of Ith gains the intrinsic ability to produce green mana. [/a]

[a]Bien sûr, tant qu'Omo reste dans les parages. La capacité d'Omo donne à Maze of Ith tous les types de terres, de la grotte à Urza et tout ce qui se trouve entre les deux, y compris la forêt. En conséquence de devenir une forêt, Maze of Ith gagne la capacité intrinsèque de produire du mana vert. [/a]

    [hr]
    [q]Does [c=Choking Vines]Vignes étrangleuses[/c] do anything against an attacker with trample such as [c=Giant Warthog]Phacochère géant[/c]?[/q]

[hr] [q]Est-ce que [cGiant Warthog]Vignes étrangleuses[/c] fait quoi que ce soit contre un attaquant avec piétinement tel que [cGiant Warthog]Phacochère géant[/c]?[/q]

    [a]Not really. While the Warthog becomes blocked due to Choking Vines, there is no creature there that's blocking it when combat damage is assigned. As we've seen last week, this means that all its damage gets assigned to the thing it's attacking, which is exactly what would have happened if you hadn't cast Choking Vines.[/a]

[a]Pas vraiment. Bien que le Warthog soit bloqué en raison de l'étouffement des Vines, il n'y a aucune créature qui le bloque lorsque des dégâts de combat sont attribués. Comme nous l'avons vu la semaine dernière, cela signifie que tous ses dégâts sont affectés à la chose qu'il attaque, ce qui est exactement ce qui se serait passé si vous n'aviez pas jeté des Vines étouffées.[/a]

    [hr]
    [q]Is putting [c=Bigger on the Inside]Plus grand à l'intérieur[/c] on [c]Pili-Pala[/c] as ridiculous as I think it is?[/q]

[hr] [q]Est-ce que mettre [cBigger à l'intérieur]Plus grand à l'intérieur[/c] sur [c]Pili-Pala[/c] est aussi ridicule que je le pense ?[/q]

    [a]If you think that this gives you an arbitrary amount of mana and cascade triggers, you're exactly right! Because of Bigger on the Inside, you can tap Pili-Pala to create two mana and to give your next spell cascade. You can use those two mana to untap Pili-Pala for its own ability to make one mana, and now you're back to the same game state where you started except for an extra mana and an extra cascade ability on your next spell. Lather, rinse, repeat as needed.[/a]

[a]Si vous pensez que cela vous donne une quantité arbitraire de déclencheurs de mana et de cascade, vous avez tout à fait raison! À cause de Bigger on the Inside, vous pouvez appuyer sur Pili-Pala pour créer deux mana et donner votre prochaine cascade de sorts.Vous pouvez utiliser ces deux manas pour extraire Pili-Pala pour sa propre capacité à faire un mana, et maintenant vous êtes de retour au même état de jeu où vous avez commencé, sauf pour un mana supplémentaire et une capacité de cascade supplémentaire sur votre prochain sort..Faire mousser, rincer, répéter au besoin.[/a]

    [hr]
    [q]Does [c=Insidious Roots]Racines insidieuses[/c]'s ability get triggered by [c=Ojer Kaslem, Deepest Growth]Ojer Kaslem, Croissance des profondeurs // Temple de la culture[/c] returning from the graveyard transformed? Or by [c=Mosswood Dreadknight // Dread Whispers]Mosswood Dreadknight[/c] being cast from my graveyard as an adventure? Or by [c=Enduring Vitality]Vitalité perpétuelle[/c] returning as an enchantment?  [/q]

[hr] [q]Est-ce que [cInsidious Roots]Racines insidieuses[/c] est déclenché par [cOjer Kaslem, Deepest Growth]Ojer Kaslem, Croissance des profondeurs // Temple de la culture[/c] revenant du cimetière transformé ?

    [a]The answer is yes for all three scenarios. In each one, you have a creature card leaving your graveyard, and that's all that Insidious Roots cares about. It doesn't matter what zone the creature card goes to or whether it's still a creature by the time it gets there.[/a]

[a]La réponse est oui pour les trois scénarios. Dans chacun d'eux, vous avez une carte de créature qui quitte votre cimetière, et c'est tout ce qui intéresse Insidious Roots. Peu importe la zone dans laquelle la carte de créature va ou si elle est toujours une créature au moment où elle y arrive.

    [hr]

[hr]

    And that's all the time we have for today. Thanks for reading, I hope you have a safe and happy Saint Patrick's Day, and please come back next week for more [b]Magic[/b] rules Q&A.

Et c'est tout le temps que nous avons pour aujourd'hui. Merci pour la lecture, j'espère que vous avez une Saint Patrick sûre et heureuse, et s'il vous plaît revenir la semaine prochaine pour plus [b]Magic [/b] règles Q & A.

    - Carsten Haese
    

- Carsten Haese

{'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'writeControl': {'requiredRevisionId': 'ALBJ4LvxbUosVaMLY5OwRTnCmG0UzOTMt_wqO6X7KEKMqWHLl5iMHMLgLqtX9feuuRG2cPErFf6epczbNDMDlQ'}, 'documentId': '1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo'

> ||@everyone|| [Notre Google doc partagé](https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire](https://www.cranial-insertion.com/article/4371) de [Cranial Insertion](https://www.cranial-insertion.com).

Cet article s'intitule "A Very Green Day", a été publié le 03/17/2025, et son ID est 4371.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab ! Code by @naereen


Starting to send the following message to my Discord server:
||@everyone|| [Notre Google doc partagé](https://docs.google.com/document/d/1DzS5Fq6iug9qnxwP0iAehLIBrTM8iQL50-AKO8REaDo/edit?usp=sharing) a été mis à jour, avec le brouillon de traduction pour [le nouvel article hebdomadaire](https://www.cranial-insertion.com/article/4371) de [Cranial Insertion](https://www.cranial-insertion.com).

Cet article s'intitule "A Very Green Day", a été publié le 03/17/2025, et son ID est 4371.

> Envoyé automatiquement depuis un notebook Python executé sur Google Colab ! Code by @naereen

Success!
CPU times: user 10min 20s, sys: 3.52 s, total: 10min 24s
Wall time: 10min 42s


''

This works fine! It takes about 12 to 15 minutes and does almost everything I wanted, great!

------------

## TODO Try to finetune [Helsinki-NLP/opus-mt-tc-big-en-fr](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr) for Magic: the Gathering keywords, vocabulary and cardnames

If I have more time, I want to:

- create a private datasets containing all the questions/answers in all the previously well-translated articles of Cranial Insertion (since 2012 it's translated in French on a weekly basis)
- fine-tune the Helsinki-NLP's LLM model on this corpus, to try to improve its performance on our future Cranial Insertion articles

If this new model indeed turns out to be "smarter" (more efficient and more precise), I will of course upload its weights to a [public model on my HuggingFace profile @Naereen](https://huggingface.co/Naereen).

If it works fine, we could use it when we will start working on translating the Magic: the Gathering Comprehensive Rules (CR), IPG, JAR, MTR.

See:
- <https://huggingface.co/docs/transformers/tasks/translation>
- <https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation-tf.ipynb>

In [108]:
assert(running_on_GPU)

AssertionError: 

This part can only run on a GPU otherwise it would be too slow!